In [ ]:
import pandas as _hex_pandas
import datetime as _hex_datetime
import json as _hex_json

In [ ]:
hex_scheduled = _hex_json.loads("false")

In [ ]:
hex_user_email = _hex_json.loads("\"example-user@example.com\"")

In [ ]:
hex_user_attributes = _hex_json.loads("{}")

In [ ]:
hex_run_context = _hex_json.loads("\"logic\"")

In [ ]:
hex_timezone = _hex_json.loads("\"UTC\"")

In [ ]:
hex_project_id = _hex_json.loads("\"dac917e3-bc06-472a-a214-167419a9fa8c\"")

In [ ]:
hex_project_name = _hex_json.loads("\"Throw No Shade: Closed Blinds Lower Temperatures \"")

In [ ]:
hex_status = _hex_json.loads("\"Completed\"")

In [ ]:
hex_categories = _hex_json.loads("[\"Deep dive\"]")

In [ ]:
hex_color_palette = _hex_json.loads("[\"#4C78A8\",\"#F58518\",\"#E45756\",\"#72B7B2\",\"#54A24B\",\"#EECA3B\",\"#B279A2\",\"#FF9DA6\",\"#9D755D\",\"#BAB0AC\"]")

## Does Closing Shades or Blinds Really Matter?
<br>Yes! A lot.
<br><br>
After conducing my very own <i>scientific</i> experiment (complete with "sensors" and "extension cords"), I discovered that during afternoons with an outdoor temperature above 80.0°F:
* With the shades closed, on average, the temperature was 4.6°F cooler and the heat index was 6.7°F lower than outdoors.
* But with the blinds open, the temperature was only improved by 1.2°F, and the heat index decrease was only 2.0°F, compared to outside.
* At the peak outdoor temperature of 95.0°F and heat index of 106°F, the closed shades provided a whopping 14.2°F reduction in temperature and a heat index slashed by 23.1°F.

<img src="https://www.jessbodie.com/img/projects/proj_shades2.png" alt="Shades Open and Closed" width="600">     

## The Quest for Enlightenment
My bedroom boasts three gorgeous, oversized windows that soak up southern sunlight.  I LOVE the sun shining through these windows Fall through Spring.  However. In the summer, my blackout cellular blinds remain closed day and night.

For two weeks this summer, I embarked on a quest to determine just how much of an impact my beloved blackout cellular blinds had on room temperature. 
  - First, I kept the shades closed 24/7.  
  - Next, I enlisted two Arduino sensors from Adafruit to record hourly temperature and humidity.  
     - The "interior shady" sensor was perched in the middle of my bedroom--always shady.  
     - The "interior sunny" sensor was taped to the interior of the window, on the sunny side of the shades, basking in all that glorious sunlight.   
<img src="https://www.jessbodie.com/img/projects/proj_shades.png" alt="Sample Image" width="200">     
     - To add some extra flair to my analysis, I also retrieved local outdoor temperature and humidity data from Open-Meteo. 
     - And yes, most nights we used the A/C. This was accounted for in the data analysis. 


In [ ]:
import pandas as pd
import pytz
import json

In [ ]:
# Data was captured from Arduino Adafruit Temperature and Humidity Sensors in July 2024
# Import saved sensor data from CSVs

# Compare Sensor - Interior-Sunny - Between the window and shades, in the path of direct sunlight
hourly_data_compare_csv = 'raw_sensor-2024-08-13_compare.csv'
hourly_data_compare = pd.read_csv(hourly_data_compare_csv)
hourly_data_compare['hour'] = pd.to_datetime(hourly_data_compare['hour']) # Convert to datetime

# Control Sensor - Interior-Shady - in middle of room, always in shade
hourly_data_csv = 'raw_sensor-2024-08-13_control.csv'
hourly_data = pd.read_csv(hourly_data_csv)
hourly_data['hour'] = pd.to_datetime(hourly_data['hour']) # Convert to datetime


In [ ]:
# Convert times to Standardized ET Format
def convert_time_ET(times):

    new_times = pd.to_datetime(times)
    if isinstance(new_times, pd.Series):
        if new_times.dt.tz is None:
            new_times = times.dt.tz_localize('UTC')
        new_times = new_times.dt.tz_convert('US/Eastern')
    else:    
        if new_times.tzinfo is None:
            new_times = times.tz_localize('UTC')

        new_times = new_times.tz_convert('US/Eastern')

    return new_times

# Identify the common start and end times for both data sets
hourly_compare_start = hourly_data_compare['hour'].min()
hourly_compare_end = hourly_data_compare['hour'].max()
hourly_start = hourly_data['hour'].min()
hourly_end = hourly_data['hour'].max()
time_range_start = max(hourly_compare_start, hourly_start)
time_range_end = min(hourly_compare_end, hourly_end)

time_range_start_ET = convert_time_ET(time_range_start)
time_range_end_ET = convert_time_ET(time_range_end)


In [ ]:
# Reduce and clarify data from sensors
def simplify_sql_to_df(query_results, display): 
    simplifed_df = pd.DataFrame(query_results)
    simplifed_df = simplifed_df[['hour', 'Temperature (C) - median', 'Relative Humidity (%) - median', 'Battery (%) - median']]
    simplifed_df = simplifed_df.rename(
        columns={'Temperature (C) - median': f'{display} Temp (C)', 
        'Relative Humidity (%) - median': f'{display} Humidity (RH%)', 
        'Battery (%) - median':f'{display} Battery (%)'}
        )
    return simplifed_df

# Create merged data frame of Control and Compare data from reduced data sets
temp_hum_control_df = simplify_sql_to_df(hourly_data, 'Control')
temp_hum_compare_df = simplify_sql_to_df(hourly_data_compare, 'Compare')

temp_hum_combined_df = pd.merge(
    temp_hum_control_df[['hour', 'Control Temp (C)', 'Control Humidity (RH%)']], 
    temp_hum_compare_df[['hour', 'Compare Temp (C)', 'Compare Humidity (RH%)']], 
    on='hour'
)

temp_hum_combined_df

,hour,Control Temp (C),Control Humidity (RH%),Compare Temp (C),Compare Humidity (RH%)
0,2024-08-12 23:00:00,26.6861,63.14750,27.0016,60.58000
1,2024-08-12 22:00:00,26.7066,63.28630,27.6711,60.02550
2,2024-08-12 21:00:00,26.6605,63.03400,27.9596,58.73470
3,2024-08-12 20:00:00,26.7322,63.97430,27.8334,60.03240
4,2024-08-12 19:00:00,26.6375,63.76435,28.6807,57.79890
...,...,...,...,...,...
426,2024-07-26 01:00:00,23.4832,65.44050,24.3643,62.74220
427,2024-07-26 00:00:00,26.7988,66.40720,27.1510,64.59110
428,2024-07-25 23:00:00,26.8143,67.26750,27.4702,64.61885
429,2024-07-25 22:00:00,26.8553,69.44110,27.8668,66.21240


In [ ]:
convert_temp_hum_df = temp_hum_combined_df.copy()

# Convert times to ET
convert_temp_hum_df['Hour (ET)'] = convert_time_ET(convert_temp_hum_df['hour'])    

# Convert temperature from Celsius to Fahrenheit
def convert_temp_F(temperatures):
    new_temperatures = (temperatures * 1.8) + 32
    return new_temperatures

convert_temp_hum_df['Control Temp (F)'] = convert_temp_F(convert_temp_hum_df['Control Temp (C)'])
convert_temp_hum_df['Compare Temp (F)'] = convert_temp_F(convert_temp_hum_df['Compare Temp (C)'])
convert_temp_hum_df


,hour,Control Temp (C),Control Humidity (RH%),Compare Temp (C),Compare Humidity (RH%),Hour (ET),Control Temp (F),Compare Temp (F)
0,2024-08-12 23:00:00,26.6861,63.14750,27.0016,60.58000,2024-08-12 19:00:00-04:00,80.03498,80.60288
1,2024-08-12 22:00:00,26.7066,63.28630,27.6711,60.02550,2024-08-12 18:00:00-04:00,80.07188,81.80798
2,2024-08-12 21:00:00,26.6605,63.03400,27.9596,58.73470,2024-08-12 17:00:00-04:00,79.98890,82.32728
3,2024-08-12 20:00:00,26.7322,63.97430,27.8334,60.03240,2024-08-12 16:00:00-04:00,80.11796,82.10012
4,2024-08-12 19:00:00,26.6375,63.76435,28.6807,57.79890,2024-08-12 15:00:00-04:00,79.94750,83.62526
...,...,...,...,...,...,...,...,...
426,2024-07-26 01:00:00,23.4832,65.44050,24.3643,62.74220,2024-07-25 21:00:00-04:00,74.26976,75.85574
427,2024-07-26 00:00:00,26.7988,66.40720,27.1510,64.59110,2024-07-25 20:00:00-04:00,80.23784,80.87180
428,2024-07-25 23:00:00,26.8143,67.26750,27.4702,64.61885,2024-07-25 19:00:00-04:00,80.26574,81.44636
429,2024-07-25 22:00:00,26.8553,69.44110,27.8668,66.21240,2024-07-25 18:00:00-04:00,80.33954,82.16024


In [ ]:
# Get historical local weather data from Open-Meteo
lat_api = 40.62993477126549
lon_api = -74.03587065767233

# While experiment was running, and sesnor data was recorded hourly, comparable weather data from public API, Open-Meteo, was retrieved
# import requests
# date_start_API = time_range_start_ET.date()
# date_end_API = time_range_end_ET.date()
# url = f'https://api.open-meteo.com/v1/forecast?latitude={lat_api}&longitude={lon_api}&hourly=temperature_2m,relative_humidity_2m&temperature_unit=fahrenheit&timezone=UTC&start_date={date_start_API}&end_date={date_end_API}'
# response = requests.get(url)
# if response.status_code == 200:
#     json_data = response.json()
#     open_meteo_df = pd.DataFrame(json_data)       
#     # Save the JSON data to a file
#     with open('open-meteo_data.json', 'w') as file:
#         json.dump(json_data, file, indent=4)
# else:
#     print(f"Error: {response.status_code}")

# Since data is historical, use downloaded JSON data
json_dl_file = open('open-meteo_data_2024-08-13.json')
json_dl_data = json.load(json_dl_file)
open_meteo_df = pd.DataFrame(json_dl_data)

open_meteo_df

,latitude,longitude,generationtime_ms,utc_offset_seconds,timezone,timezone_abbreviation,elevation,hourly_units,hourly
time,40.632072,-74.02001,0.1719,0,UTC,UTC,15.0,iso8601,"[2024-07-25T00:00, 2024-07-25T01:00, 2024-07-2..."
temperature_2m,40.632072,-74.02001,0.1719,0,UTC,UTC,15.0,°F,"[78.0, 76.7, 76.4, 75.8, 74.6, 74.1, 74.6, 74...."
relative_humidity_2m,40.632072,-74.02001,0.1719,0,UTC,UTC,15.0,%,"[79, 80, 81, 90, 89, 90, 92, 91, 92, 94, 93, 9..."


In [ ]:
# Reduce and flatten Open-Meteo Data
open_meteo_df_initial = pd.DataFrame(open_meteo_df['hourly'])

simple_open_meteo_df = pd.DataFrame({
    'time': open_meteo_df_initial['hourly']['time'],
    'temperature_2m': open_meteo_df_initial['hourly']['temperature_2m'],
    'relative_humidity_2m': open_meteo_df_initial['hourly']['relative_humidity_2m']
})

simple_open_meteo_df['Hour (ET)'] = convert_time_ET(pd.to_datetime(simple_open_meteo_df['time']))

simple_open_meteo_df

,time,temperature_2m,relative_humidity_2m,Hour (ET)
0,2024-07-25T00:00,78.0,79,2024-07-24 20:00:00-04:00
1,2024-07-25T01:00,76.7,80,2024-07-24 21:00:00-04:00
2,2024-07-25T02:00,76.4,81,2024-07-24 22:00:00-04:00
3,2024-07-25T03:00,75.8,90,2024-07-24 23:00:00-04:00
4,2024-07-25T04:00,74.6,89,2024-07-25 00:00:00-04:00
...,...,...,...,...
739,2024-08-24T19:00,80.4,47,2024-08-24 15:00:00-04:00
740,2024-08-24T20:00,78.7,52,2024-08-24 16:00:00-04:00
741,2024-08-24T21:00,77.4,57,2024-08-24 17:00:00-04:00
742,2024-08-24T22:00,77.3,61,2024-08-24 18:00:00-04:00


## Raw Data: Temperature & Humidity

In [ ]:
# Merge the Open-Meteo temp and humidity with the already-merged control and compare temp and humidity data
temp_hum_w_public_df = pd.merge(
    convert_temp_hum_df[['Hour (ET)', 'Control Temp (F)', 'Compare Temp (F)', 'Control Humidity (RH%)', 'Compare Humidity (RH%)']], 
    simple_open_meteo_df[['Hour (ET)', 'temperature_2m', 'relative_humidity_2m']], 
    on='Hour (ET)'
)
temp_hum_w_public_df['Date/Hour'] = temp_hum_w_public_df['Hour (ET)']
temp_hum_w_public_df['Hour (ET)'] = temp_hum_w_public_df['Hour (ET)'].dt.tz_localize(None)
temp_hum_w_public_df = temp_hum_w_public_df.rename(columns={'temperature_2m':'Outdoor Temp (F)', 'relative_humidity_2m': 'Outdoor Humidity (RH%)'})

temp_hum_w_public_df

,Hour (ET),Control Temp (F),Compare Temp (F),Control Humidity (RH%),Compare Humidity (RH%),Outdoor Temp (F),Outdoor Humidity (RH%),Date/Hour
0,2024-08-12 19:00:00,80.03498,80.60288,63.14750,60.58000,79.4,42,2024-08-12 19:00:00-04:00
1,2024-08-12 18:00:00,80.07188,81.80798,63.28630,60.02550,80.5,46,2024-08-12 18:00:00-04:00
2,2024-08-12 17:00:00,79.98890,82.32728,63.03400,58.73470,78.0,57,2024-08-12 17:00:00-04:00
3,2024-08-12 16:00:00,80.11796,82.10012,63.97430,60.03240,80.5,49,2024-08-12 16:00:00-04:00
4,2024-08-12 15:00:00,79.94750,83.62526,63.76435,57.79890,82.6,43,2024-08-12 15:00:00-04:00
...,...,...,...,...,...,...,...,...
426,2024-07-25 21:00:00,74.26976,75.85574,65.44050,62.74220,79.2,48,2024-07-25 21:00:00-04:00
427,2024-07-25 20:00:00,80.23784,80.87180,66.40720,64.59110,81.1,47,2024-07-25 20:00:00-04:00
428,2024-07-25 19:00:00,80.26574,81.44636,67.26750,64.61885,82.1,48,2024-07-25 19:00:00-04:00
429,2024-07-25 18:00:00,80.33954,82.16024,69.44110,66.21240,82.2,52,2024-07-25 18:00:00-04:00


In [ ]:
# Calculate the Heat Index, per NOAA: National Weather Service
# If the Heat Index is above 80 degrees, use this formula:
# HI = 0.5 * {T + 61.0 + [(T-68.0)*1.2] + (RH*0.094)}
# If the Heat Index is below 80 degrees, use this formula:
# HI = -42.379 + 2.04901523*T + 10.14333127*RH - .22475541*T*RH - .00683783*T*T - .05481717*RH*RH + .00122874*T*T*RH + .00085282*T*RH*RH - .00000199*T*T*RH*RH

def calc_heat_index(temp, rh):
    heat_index = 0.5 * (temp + 61.0 + ((temp-68.0)*1.2) + (rh*0.094))
    if (heat_index <= 80):
        return round(heat_index, 1) 
    else:
        heat_index = -42.379 + 2.04901523*temp + 10.14333127*rh - .22475541*temp*rh - .00683783*temp*temp - .05481717*rh*rh + .00122874*temp*temp*rh + .00085282*temp*rh*rh - .00000199*temp*temp*rh*rh
        return round(heat_index, 1) 

# Calculate the Heat Index for inside, behind the shades
temp_hum_w_public_df['Control Heat Index (F)'] = temp_hum_w_public_df.apply(
    lambda row: calc_heat_index(row['Control Temp (F)'], row['Control Humidity (RH%)']), 
    axis=1
)

# Calculate the Heat Index for inside, in the sun
temp_hum_w_public_df['Compare Heat Index (F)'] = temp_hum_w_public_df.apply(
    lambda row: calc_heat_index(row['Compare Temp (F)'], row['Compare Humidity (RH%)']), 
    axis=1
)

# Calculate the Heat Index for outdoors
temp_hum_w_public_df['Outdoor Heat Index (F)'] = temp_hum_w_public_df.apply(
    lambda row: calc_heat_index(row['Outdoor Temp (F)'], row['Outdoor Humidity (RH%)']), 
    axis=1
)

temp_hum_w_public_df.head(20)

,Hour (ET),Control Temp (F),Compare Temp (F),Control Humidity (RH%),Compare Humidity (RH%),Outdoor Temp (F),Outdoor Humidity (RH%),Date/Hour,Control Heat Index (F),Compare Heat Index (F),Outdoor Heat Index (F)
0,2024-08-12 19:00:00,80.03498,80.60288,63.14750,60.58000,79.4,42,2024-08-12 19:00:00-04:00,82.2,82.6,79.0
1,2024-08-12 18:00:00,80.07188,81.80798,63.28630,60.02550,80.5,46,2024-08-12 18:00:00-04:00,82.3,84.2,80.8
2,2024-08-12 17:00:00,79.98890,82.32728,63.03400,58.73470,78.0,57,2024-08-12 17:00:00-04:00,82.1,84.7,78.2
3,2024-08-12 16:00:00,80.11796,82.10012,63.97430,60.03240,80.5,49,2024-08-12 16:00:00-04:00,82.4,84.6,81.2
4,2024-08-12 15:00:00,79.94750,83.62526,63.76435,57.79890,82.6,43,2024-08-12 15:00:00-04:00,82.2,86.4,82.4
5,2024-08-12 14:00:00,79.31534,85.28486,63.62020,54.75810,82.3,42,2024-08-12 14:00:00-04:00,79.9,88.2,82.0
6,2024-08-12 13:00:00,78.56834,84.94178,63.37250,53.83590,81.0,46,2024-08-12 13:00:00-04:00,79.1,87.4,81.3
7,2024-08-12 12:00:00,77.84870,82.63787,63.03260,55.33435,79.7,51,2024-08-12 12:00:00-04:00,78.3,84.4,79.8
8,2024-08-12 11:00:00,77.24930,78.84122,63.19200,60.13190,77.2,52,2024-08-12 11:00:00-04:00,77.6,79.3,77.1
9,2024-08-12 10:00:00,76.38674,77.65466,64.09520,62.65190,74.7,55,2024-08-12 10:00:00-04:00,76.7,78.1,74.5


In [ ]:
outdoor_temp_max = temp_hum_w_public_df['Outdoor Temp (F)'].max()
outdoor_HI_max = temp_hum_w_public_df['Outdoor Heat Index (F)'].max()


In [ ]:
# To better interpret data, detail when A/C is in use

# Estimate that A/C goes on at approximately 8pm and off at 7am, specify overnight exception
ac_start_time = pd.to_datetime('20:00:00').time()
ac_end_time = pd.to_datetime('07:00:00').time()
ac_start_away = pd.to_datetime('2024-07-26 16:00:00')
ac_end_away =  pd.to_datetime('2024-07-27 20:00:00')

def ac_is_on(date_time):
    eval_date_time = pd.to_datetime(date_time)
    eval_time = eval_date_time.time()
    if ((ac_start_time <= eval_time or eval_time <= ac_end_time) and (ac_start_away >= eval_date_time or eval_date_time >= ac_end_away)):
        return outdoor_HI_max
    return 0

temp_hum_w_public_df['A/C Is On'] = temp_hum_w_public_df['Hour (ET)'].apply(ac_is_on)

# Reorder columns for readability
new_col_order = ['Hour (ET)','Control Temp (F)','Compare Temp (F)','Outdoor Temp (F)','Control Humidity (RH%)','Compare Humidity (RH%)','Outdoor Humidity (RH%)','Control Heat Index (F)','Compare Heat Index (F)','Outdoor Heat Index (F)','A/C Is On','Date/Hour']
temp_hum_w_public_df = temp_hum_w_public_df[new_col_order]

temp_hum_w_public_df

,Hour (ET),Control Temp (F),Compare Temp (F),Outdoor Temp (F),Control Humidity (RH%),Compare Humidity (RH%),Outdoor Humidity (RH%),Control Heat Index (F),Compare Heat Index (F),Outdoor Heat Index (F),A/C Is On,Date/Hour
0,2024-08-12 19:00:00,80.03498,80.60288,79.4,63.14750,60.58000,42,82.2,82.6,79.0,0.0,2024-08-12 19:00:00-04:00
1,2024-08-12 18:00:00,80.07188,81.80798,80.5,63.28630,60.02550,46,82.3,84.2,80.8,0.0,2024-08-12 18:00:00-04:00
2,2024-08-12 17:00:00,79.98890,82.32728,78.0,63.03400,58.73470,57,82.1,84.7,78.2,0.0,2024-08-12 17:00:00-04:00
3,2024-08-12 16:00:00,80.11796,82.10012,80.5,63.97430,60.03240,49,82.4,84.6,81.2,0.0,2024-08-12 16:00:00-04:00
4,2024-08-12 15:00:00,79.94750,83.62526,82.6,63.76435,57.79890,43,82.2,86.4,82.4,0.0,2024-08-12 15:00:00-04:00
...,...,...,...,...,...,...,...,...,...,...,...,...
426,2024-07-25 21:00:00,74.26976,75.85574,79.2,65.44050,62.74220,48,74.5,76.1,79.1,107.8,2024-07-25 21:00:00-04:00
427,2024-07-25 20:00:00,80.23784,80.87180,81.1,66.40720,64.59110,47,82.9,83.6,81.5,107.8,2024-07-25 20:00:00-04:00
428,2024-07-25 19:00:00,80.26574,81.44636,82.1,67.26750,64.61885,48,83.0,84.4,82.6,0.0,2024-07-25 19:00:00-04:00
429,2024-07-25 18:00:00,80.33954,82.16024,82.2,69.44110,66.21240,52,83.4,85.9,83.3,0.0,2024-07-25 18:00:00-04:00


## Visualizing the Findings: Temperature & Humidity

In [ ]:
import altair
chart_temp_hum_w_public_df = altair.Chart.from_json(r"""
{
    "width": "container",
    "height": "container",
    "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
    "layer": [
        {
            "description": "outer data layer",
            "resolve": {
                "scale": {
                    "color": "independent",
                    "y": "shared"
                }
            },
            "layer": [
                {
                    "description": "line series layer",
                    "transform": [],
                    "layer": [
                        {
                            "description": "line mark layer",
                            "mark": {
                                "type": "line",
                                "point": false,
                                "clip": true,
                                "strokeWidth": 3,
                                "strokeCap": "square",
                                "strokeJoin": "round",
                                "cursor": "crosshair"
                            },
                            "encoding": {
                                "opacity": {
                                    "value": 1
                                }
                            }
                        },
                        {
                            "description": "pivot tooltip point mark layer",
                            "mark": {
                                "type": "point",
                                "tooltip": false,
                                "clip": true
                            },
                            "encoding": {
                                "opacity": {
                                    "value": 0,
                                    "condition": {
                                        "param": "shared_pivot_hover_param",
                                        "value": 1,
                                        "empty": false
                                    }
                                },
                                "size": {
                                    "value": 80
                                }
                            }
                        }
                    ],
                    "encoding": {
                        "x": {
                            "field": "Hour (ET)",
                            "type": "temporal",
                            "title": "",
                            "scale": {},
                            "axis": {
                                "grid": true,
                                "ticks": true,
                                "labels": true,
                                "labelFlush": false
                            }
                        },
                        "y": {
                            "field": "Control Temp (F)",
                            "type": "quantitative",
                            "title": "Temperature (F)",
                            "scale": {
                                "domainMin": 64
                            },
                            "axis": {
                                "grid": true,
                                "ticks": true,
                                "labels": true,
                                "labelFlush": false,
                                "format": {
                                    "format": "NUMBER",
                                    "currency": "USD",
                                    "columnType": "NUMBER",
                                    "showSeparators": false,
                                    "numDecimalDigits": -1,
                                    "abbreviateLargeNumbers": true,
                                    "nanFormat": ""
                                },
                                "formatType": "NUMBER_FORMATTER"
                            }
                        },
                        "color": {
                            "datum": "Interior Shady Temp (F)",
                            "scale": {
                                "range": [
                                    "#4C78A8",
                                    "#4C78A8"
                                ]
                            },
                            "legend": {
                                "symbolOpacity": 1,
                                "symbolType": "stroke"
                            }
                        }
                    }
                },
                {
                    "description": "line series layer",
                    "transform": [],
                    "layer": [
                        {
                            "description": "line mark layer",
                            "mark": {
                                "type": "line",
                                "point": false,
                                "clip": true,
                                "strokeWidth": 3,
                                "strokeCap": "square",
                                "strokeJoin": "round",
                                "cursor": "crosshair"
                            },
                            "encoding": {
                                "opacity": {
                                    "value": 1
                                }
                            }
                        },
                        {
                            "description": "pivot tooltip point mark layer",
                            "mark": {
                                "type": "point",
                                "tooltip": false,
                                "clip": true
                            },
                            "encoding": {
                                "opacity": {
                                    "value": 0,
                                    "condition": {
                                        "param": "shared_pivot_hover_param",
                                        "value": 1,
                                        "empty": false
                                    }
                                },
                                "size": {
                                    "value": 80
                                }
                            }
                        }
                    ],
                    "encoding": {
                        "x": {
                            "field": "Hour (ET)",
                            "type": "temporal",
                            "title": "",
                            "scale": {},
                            "axis": {
                                "grid": true,
                                "ticks": true,
                                "labels": true,
                                "labelFlush": false
                            }
                        },
                        "y": {
                            "field": "Compare Temp (F)",
                            "type": "quantitative",
                            "title": "Temperature (F)",
                            "scale": {
                                "domainMin": 64
                            },
                            "axis": {
                                "grid": true,
                                "ticks": true,
                                "labels": true,
                                "labelFlush": false,
                                "format": {
                                    "format": "NUMBER",
                                    "currency": "USD",
                                    "columnType": "NUMBER",
                                    "showSeparators": false,
                                    "numDecimalDigits": -1,
                                    "abbreviateLargeNumbers": true,
                                    "nanFormat": ""
                                },
                                "formatType": "NUMBER_FORMATTER"
                            }
                        },
                        "color": {
                            "datum": "Interior Sunny Temp (F)",
                            "scale": {
                                "range": [
                                    "#F58518",
                                    "#F58518"
                                ]
                            },
                            "legend": {
                                "symbolOpacity": 1,
                                "symbolType": "stroke"
                            }
                        }
                    }
                },
                {
                    "description": "line series layer",
                    "transform": [],
                    "layer": [
                        {
                            "description": "line mark layer",
                            "mark": {
                                "type": "line",
                                "point": false,
                                "clip": true,
                                "strokeWidth": 3,
                                "strokeCap": "square",
                                "strokeJoin": "round",
                                "cursor": "crosshair"
                            },
                            "encoding": {
                                "opacity": {
                                    "value": 1
                                }
                            }
                        },
                        {
                            "description": "pivot tooltip point mark layer",
                            "mark": {
                                "type": "point",
                                "tooltip": false,
                                "clip": true
                            },
                            "encoding": {
                                "opacity": {
                                    "value": 0,
                                    "condition": {
                                        "param": "shared_pivot_hover_param",
                                        "value": 1,
                                        "empty": false
                                    }
                                },
                                "size": {
                                    "value": 80
                                }
                            }
                        }
                    ],
                    "encoding": {
                        "x": {
                            "field": "Hour (ET)",
                            "type": "temporal",
                            "title": "",
                            "scale": {},
                            "axis": {
                                "grid": true,
                                "ticks": true,
                                "labels": true,
                                "labelFlush": false
                            }
                        },
                        "y": {
                            "field": "Outdoor Temp (F)",
                            "type": "quantitative",
                            "title": "Temperature (F)",
                            "scale": {
                                "domainMin": 64
                            },
                            "axis": {
                                "grid": true,
                                "ticks": true,
                                "labels": true,
                                "labelFlush": false,
                                "format": {
                                    "format": "NUMBER",
                                    "currency": "USD",
                                    "columnType": "NUMBER",
                                    "showSeparators": false,
                                    "numDecimalDigits": -1,
                                    "abbreviateLargeNumbers": true,
                                    "nanFormat": ""
                                },
                                "formatType": "NUMBER_FORMATTER"
                            }
                        },
                        "color": {
                            "datum": "Outdoor Temp (F)",
                            "scale": {
                                "range": [
                                    "#E45756",
                                    "#E45756"
                                ]
                            },
                            "legend": {
                                "symbolOpacity": 1,
                                "symbolType": "stroke"
                            }
                        }
                    }
                },
                {
                    "description": "area series layer",
                    "transform": [],
                    "layer": [
                        {
                            "description": "background area mark layer",
                            "mark": {
                                "type": "area",
                                "interpolate": "step",
                                "point": false,
                                "line": true,
                                "strokeJoin": "round",
                                "clip": true,
                                "cursor": "crosshair"
                            },
                            "encoding": {
                                "color": {
                                    "datum": "A/C Is On",
                                    "scale": {
                                        "range": [
                                            "#4C78A8",
                                            "#4C78A8"
                                        ]
                                    },
                                    "legend": {
                                        "symbolOpacity": 0.15
                                    }
                                },
                                "opacity": {
                                    "value": 0.2
                                }
                            }
                        },
                        {
                            "transform": [],
                            "description": "area mark layer",
                            "mark": {
                                "type": "area",
                                "interpolate": "step",
                                "point": false,
                                "line": true,
                                "clip": true
                            },
                            "encoding": {
                                "color": {
                                    "datum": "A/C Is On",
                                    "scale": {
                                        "range": [
                                            "#4C78A8",
                                            "#4C78A8"
                                        ]
                                    },
                                    "legend": {
                                        "symbolOpacity": 0.15
                                    }
                                },
                                "opacity": {
                                    "value": 0.15
                                }
                            }
                        },
                        {
                            "description": "pivot tooltip point mark layer",
                            "mark": {
                                "type": "point",
                                "tooltip": false,
                                "clip": true
                            },
                            "encoding": {
                                "opacity": {
                                    "value": 0,
                                    "condition": {
                                        "param": "shared_pivot_hover_param",
                                        "value": 1,
                                        "empty": false
                                    }
                                },
                                "size": {
                                    "value": 80
                                },
                                "color": {
                                    "datum": "A/C Is On",
                                    "scale": {
                                        "range": [
                                            "#4C78A8",
                                            "#4C78A8"
                                        ]
                                    },
                                    "legend": {
                                        "symbolOpacity": 0.15
                                    }
                                },
                                "y": {
                                    "field": "A/C Is On",
                                    "type": "quantitative",
                                    "title": "Temperature (F)",
                                    "scale": {
                                        "domainMin": 64
                                    },
                                    "axis": {
                                        "grid": true,
                                        "ticks": true,
                                        "labels": true,
                                        "labelFlush": false,
                                        "format": {
                                            "format": "NUMBER",
                                            "currency": "USD",
                                            "columnType": "NUMBER",
                                            "showSeparators": false,
                                            "numDecimalDigits": -1,
                                            "abbreviateLargeNumbers": true,
                                            "nanFormat": ""
                                        },
                                        "formatType": "NUMBER_FORMATTER"
                                    },
                                    "stack": "zero"
                                }
                            }
                        }
                    ],
                    "encoding": {
                        "x": {
                            "field": "Hour (ET)",
                            "type": "temporal",
                            "title": "",
                            "scale": {},
                            "axis": {
                                "grid": true,
                                "ticks": true,
                                "labels": true,
                                "labelFlush": false
                            }
                        },
                        "y": {
                            "field": "A/C Is On",
                            "type": "quantitative",
                            "title": "Temperature (F)",
                            "scale": {
                                "domainMin": 64
                            },
                            "axis": {
                                "grid": true,
                                "ticks": true,
                                "labels": true,
                                "labelFlush": false,
                                "format": {
                                    "format": "NUMBER",
                                    "currency": "USD",
                                    "columnType": "NUMBER",
                                    "showSeparators": false,
                                    "numDecimalDigits": -1,
                                    "abbreviateLargeNumbers": true,
                                    "nanFormat": ""
                                },
                                "formatType": "NUMBER_FORMATTER"
                            }
                        }
                    }
                },
                {
                    "description": "shared tooltip layer",
                    "encoding": {
                        "x": {
                            "field": "Hour (ET)",
                            "type": "temporal",
                            "title": "",
                            "scale": {},
                            "axis": {
                                "grid": true,
                                "ticks": true,
                                "labels": true,
                                "labelFlush": false
                            }
                        }
                    },
                    "layer": [
                        {
                            "description": "shared tooltip rule mark layer",
                            "mark": {
                                "type": "rule",
                                "clip": true
                            },
                            "encoding": {
                                "opacity": {
                                    "condition": {
                                        "value": 0.3,
                                        "param": "shared_pivot_hover_param",
                                        "empty": false
                                    },
                                    "value": 0
                                },
                                "color": {
                                    "value": "CHART_DEFAULT_RULE_COLOR_MARKER"
                                },
                                "tooltip": [
                                    {
                                        "field": "Hour (ET)",
                                        "type": "temporal",
                                        "title": " "
                                    },
                                    {
                                        "field": "Control Temp (F)",
                                        "type": "quantitative",
                                        "formatType": "NUMBER_FORMATTER",
                                        "format": {
                                            "format": "NUMBER",
                                            "currency": "USD",
                                            "columnType": "NUMBER",
                                            "showSeparators": false,
                                            "numDecimalDigits": -1,
                                            "abbreviateLargeNumbers": true,
                                            "nanFormat": ""
                                        },
                                        "title": "Interior Shady Temp (F)"
                                    },
                                    {
                                        "field": "Compare Temp (F)",
                                        "type": "quantitative",
                                        "formatType": "NUMBER_FORMATTER",
                                        "format": {
                                            "format": "NUMBER",
                                            "currency": "USD",
                                            "columnType": "NUMBER",
                                            "showSeparators": false,
                                            "numDecimalDigits": -1,
                                            "abbreviateLargeNumbers": true,
                                            "nanFormat": ""
                                        },
                                        "title": "Interior Sunny Temp (F)"
                                    },
                                    {
                                        "field": "Outdoor Temp (F)",
                                        "type": "quantitative",
                                        "formatType": "NUMBER_FORMATTER",
                                        "format": {
                                            "format": "NUMBER",
                                            "currency": "USD",
                                            "columnType": "NUMBER",
                                            "showSeparators": false,
                                            "numDecimalDigits": -1,
                                            "abbreviateLargeNumbers": true,
                                            "nanFormat": ""
                                        },
                                        "title": "Outdoor Temp (F)"
                                    },
                                    {
                                        "field": "A/C Is On",
                                        "type": "quantitative",
                                        "formatType": "NUMBER_FORMATTER",
                                        "format": {
                                            "format": "NUMBER",
                                            "currency": "USD",
                                            "columnType": "NUMBER",
                                            "showSeparators": false,
                                            "numDecimalDigits": -1,
                                            "abbreviateLargeNumbers": true,
                                            "nanFormat": ""
                                        },
                                        "title": "A/C Is On"
                                    }
                                ]
                            },
                            "params": [
                                {
                                    "name": "shared_pivot_hover_param",
                                    "select": {
                                        "type": "point",
                                        "fields": [
                                            "Hour (ET)"
                                        ],
                                        "nearest": true,
                                        "on": "mouseover",
                                        "clear": "mouseout"
                                    }
                                }
                            ]
                        }
                    ]
                }
            ],
            "transform": [
                {
                    "calculate": "toDate(datum[\"Hour (ET)\"])",
                    "as": "Hour (ET)"
                },
                {
                    "filter": "isValid(datum[\"Hour (ET)\"])"
                }
            ]
        }
    ],
    "config": {
        "legend": {
            "orient": "right"
        },
        "font": "\"IBM Plex Sans\", system-ui, -apple-system, BlinkMacSystemFont, sans-serif",
        "view": {}
    },
    "datasets": {
        "layer00": [
            {
                "name": "dummy",
                "value": 0
            }
        ]
    },
    "usermeta": {
        "selectionConfigs": {},
        "columnNameMappings": {}
    }
}
""")
chart_temp_hum_w_public_df.datasets.layer00 = temp_hum_w_public_df.to_json(orient='records')
chart_temp_hum_w_public_df.display(actions=False)

In [ ]:
import altair
chart_temp_hum_w_public_df = altair.Chart.from_json(r"""
{
    "width": "container",
    "height": "container",
    "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
    "layer": [
        {
            "description": "outer data layer",
            "resolve": {
                "scale": {
                    "color": "independent",
                    "y": "shared"
                }
            },
            "layer": [
                {
                    "description": "line series layer",
                    "transform": [],
                    "layer": [
                        {
                            "description": "line mark layer",
                            "mark": {
                                "type": "line",
                                "point": false,
                                "clip": true,
                                "strokeWidth": 3,
                                "strokeCap": "square",
                                "strokeJoin": "round",
                                "cursor": "crosshair"
                            },
                            "encoding": {
                                "opacity": {
                                    "value": 1
                                }
                            }
                        },
                        {
                            "description": "pivot tooltip point mark layer",
                            "mark": {
                                "type": "point",
                                "tooltip": false,
                                "clip": true
                            },
                            "encoding": {
                                "opacity": {
                                    "value": 0,
                                    "condition": {
                                        "param": "shared_pivot_hover_param",
                                        "value": 1,
                                        "empty": false
                                    }
                                },
                                "size": {
                                    "value": 80
                                }
                            }
                        }
                    ],
                    "encoding": {
                        "x": {
                            "field": "Hour (ET)",
                            "type": "temporal",
                            "title": "",
                            "scale": {},
                            "axis": {
                                "grid": true,
                                "ticks": true,
                                "labels": true,
                                "labelFlush": false
                            }
                        },
                        "y": {
                            "field": "Control Heat Index (F)",
                            "type": "quantitative",
                            "title": "Temperature (F)",
                            "scale": {
                                "domainMin": 64
                            },
                            "axis": {
                                "grid": true,
                                "ticks": true,
                                "labels": true,
                                "labelFlush": false,
                                "format": {
                                    "format": "NUMBER",
                                    "columnType": "NUMBER",
                                    "numDecimalDigits": -1,
                                    "currency": "$",
                                    "nanFormat": ""
                                },
                                "formatType": "NUMBER_FORMATTER"
                            }
                        },
                        "color": {
                            "datum": "Interior Shady Heat Index (F)",
                            "scale": {
                                "range": [
                                    "#4C78A8",
                                    "#4C78A8"
                                ]
                            },
                            "legend": {
                                "symbolOpacity": 1,
                                "symbolType": "stroke"
                            }
                        }
                    }
                },
                {
                    "description": "line series layer",
                    "transform": [],
                    "layer": [
                        {
                            "description": "line mark layer",
                            "mark": {
                                "type": "line",
                                "point": false,
                                "clip": true,
                                "strokeWidth": 3,
                                "strokeCap": "square",
                                "strokeJoin": "round",
                                "cursor": "crosshair"
                            },
                            "encoding": {
                                "opacity": {
                                    "value": 1
                                }
                            }
                        },
                        {
                            "description": "pivot tooltip point mark layer",
                            "mark": {
                                "type": "point",
                                "tooltip": false,
                                "clip": true
                            },
                            "encoding": {
                                "opacity": {
                                    "value": 0,
                                    "condition": {
                                        "param": "shared_pivot_hover_param",
                                        "value": 1,
                                        "empty": false
                                    }
                                },
                                "size": {
                                    "value": 80
                                }
                            }
                        }
                    ],
                    "encoding": {
                        "x": {
                            "field": "Hour (ET)",
                            "type": "temporal",
                            "title": "",
                            "scale": {},
                            "axis": {
                                "grid": true,
                                "ticks": true,
                                "labels": true,
                                "labelFlush": false
                            }
                        },
                        "y": {
                            "field": "Compare Heat Index (F)",
                            "type": "quantitative",
                            "title": "Temperature (F)",
                            "scale": {
                                "domainMin": 64
                            },
                            "axis": {
                                "grid": true,
                                "ticks": true,
                                "labels": true,
                                "labelFlush": false,
                                "format": {
                                    "format": "NUMBER",
                                    "columnType": "NUMBER",
                                    "numDecimalDigits": -1,
                                    "currency": "$",
                                    "nanFormat": ""
                                },
                                "formatType": "NUMBER_FORMATTER"
                            }
                        },
                        "color": {
                            "datum": "Interior Sunny Heat Index (F)",
                            "scale": {
                                "range": [
                                    "#F58518",
                                    "#F58518"
                                ]
                            },
                            "legend": {
                                "symbolOpacity": 1,
                                "symbolType": "stroke"
                            }
                        }
                    }
                },
                {
                    "description": "line series layer",
                    "transform": [],
                    "layer": [
                        {
                            "description": "line mark layer",
                            "mark": {
                                "type": "line",
                                "point": false,
                                "clip": true,
                                "strokeWidth": 3,
                                "strokeCap": "square",
                                "strokeJoin": "round",
                                "cursor": "crosshair"
                            },
                            "encoding": {
                                "opacity": {
                                    "value": 1
                                }
                            }
                        },
                        {
                            "description": "pivot tooltip point mark layer",
                            "mark": {
                                "type": "point",
                                "tooltip": false,
                                "clip": true
                            },
                            "encoding": {
                                "opacity": {
                                    "value": 0,
                                    "condition": {
                                        "param": "shared_pivot_hover_param",
                                        "value": 1,
                                        "empty": false
                                    }
                                },
                                "size": {
                                    "value": 80
                                }
                            }
                        }
                    ],
                    "encoding": {
                        "x": {
                            "field": "Hour (ET)",
                            "type": "temporal",
                            "title": "",
                            "scale": {},
                            "axis": {
                                "grid": true,
                                "ticks": true,
                                "labels": true,
                                "labelFlush": false
                            }
                        },
                        "y": {
                            "field": "Outdoor Heat Index (F)",
                            "type": "quantitative",
                            "title": "Temperature (F)",
                            "scale": {
                                "domainMin": 64
                            },
                            "axis": {
                                "grid": true,
                                "ticks": true,
                                "labels": true,
                                "labelFlush": false,
                                "format": {
                                    "format": "NUMBER",
                                    "columnType": "NUMBER",
                                    "numDecimalDigits": -1,
                                    "currency": "$",
                                    "nanFormat": ""
                                },
                                "formatType": "NUMBER_FORMATTER"
                            }
                        },
                        "color": {
                            "datum": "Outdoor Heat Index (F)",
                            "scale": {
                                "range": [
                                    "#E45756",
                                    "#E45756"
                                ]
                            },
                            "legend": {
                                "symbolOpacity": 1,
                                "symbolType": "stroke"
                            }
                        }
                    }
                },
                {
                    "description": "area series layer",
                    "transform": [],
                    "layer": [
                        {
                            "description": "background area mark layer",
                            "mark": {
                                "type": "area",
                                "interpolate": "step",
                                "point": false,
                                "line": false,
                                "strokeJoin": "round",
                                "clip": true,
                                "cursor": "crosshair"
                            },
                            "encoding": {
                                "color": {
                                    "datum": "A/C Is On",
                                    "scale": {
                                        "range": [
                                            "#4C78A8",
                                            "#4C78A8"
                                        ]
                                    },
                                    "legend": {
                                        "symbolOpacity": 0.15
                                    }
                                },
                                "opacity": {
                                    "value": 0.2
                                }
                            }
                        },
                        {
                            "transform": [],
                            "description": "area mark layer",
                            "mark": {
                                "type": "area",
                                "interpolate": "step",
                                "point": false,
                                "line": false,
                                "clip": true
                            },
                            "encoding": {
                                "color": {
                                    "datum": "A/C Is On",
                                    "scale": {
                                        "range": [
                                            "#4C78A8",
                                            "#4C78A8"
                                        ]
                                    },
                                    "legend": {
                                        "symbolOpacity": 0.15
                                    }
                                },
                                "opacity": {
                                    "value": 0.15
                                }
                            }
                        },
                        {
                            "description": "pivot tooltip point mark layer",
                            "mark": {
                                "type": "point",
                                "tooltip": false,
                                "clip": true
                            },
                            "encoding": {
                                "opacity": {
                                    "value": 0,
                                    "condition": {
                                        "param": "shared_pivot_hover_param",
                                        "value": 1,
                                        "empty": false
                                    }
                                },
                                "size": {
                                    "value": 80
                                },
                                "color": {
                                    "datum": "A/C Is On",
                                    "scale": {
                                        "range": [
                                            "#4C78A8",
                                            "#4C78A8"
                                        ]
                                    },
                                    "legend": {
                                        "symbolOpacity": 0.15
                                    }
                                },
                                "y": {
                                    "field": "A/C Is On",
                                    "type": "quantitative",
                                    "title": "Temperature (F)",
                                    "scale": {
                                        "domainMin": 64
                                    },
                                    "axis": {
                                        "grid": true,
                                        "ticks": true,
                                        "labels": true,
                                        "labelFlush": false,
                                        "format": {
                                            "format": "NUMBER",
                                            "columnType": "NUMBER",
                                            "numDecimalDigits": -1,
                                            "currency": "$",
                                            "nanFormat": ""
                                        },
                                        "formatType": "NUMBER_FORMATTER"
                                    },
                                    "stack": "zero"
                                }
                            }
                        }
                    ],
                    "encoding": {
                        "x": {
                            "field": "Hour (ET)",
                            "type": "temporal",
                            "title": "",
                            "scale": {},
                            "axis": {
                                "grid": true,
                                "ticks": true,
                                "labels": true,
                                "labelFlush": false
                            }
                        },
                        "y": {
                            "field": "A/C Is On",
                            "type": "quantitative",
                            "title": "Temperature (F)",
                            "scale": {
                                "domainMin": 64
                            },
                            "axis": {
                                "grid": true,
                                "ticks": true,
                                "labels": true,
                                "labelFlush": false,
                                "format": {
                                    "format": "NUMBER",
                                    "columnType": "NUMBER",
                                    "numDecimalDigits": -1,
                                    "currency": "$",
                                    "nanFormat": ""
                                },
                                "formatType": "NUMBER_FORMATTER"
                            }
                        }
                    }
                },
                {
                    "description": "shared tooltip layer",
                    "encoding": {
                        "x": {
                            "field": "Hour (ET)",
                            "type": "temporal",
                            "title": "",
                            "scale": {},
                            "axis": {
                                "grid": true,
                                "ticks": true,
                                "labels": true,
                                "labelFlush": false
                            }
                        }
                    },
                    "layer": [
                        {
                            "description": "shared tooltip rule mark layer",
                            "mark": {
                                "type": "rule",
                                "clip": true
                            },
                            "encoding": {
                                "opacity": {
                                    "condition": {
                                        "value": 0.3,
                                        "param": "shared_pivot_hover_param",
                                        "empty": false
                                    },
                                    "value": 0
                                },
                                "color": {
                                    "value": "CHART_DEFAULT_RULE_COLOR_MARKER"
                                },
                                "tooltip": [
                                    {
                                        "field": "Hour (ET)",
                                        "type": "temporal",
                                        "title": " "
                                    },
                                    {
                                        "field": "Control Heat Index (F)",
                                        "type": "quantitative",
                                        "formatType": "NUMBER_FORMATTER",
                                        "format": {
                                            "format": "NUMBER",
                                            "columnType": "NUMBER",
                                            "numDecimalDigits": -1,
                                            "currency": "$",
                                            "nanFormat": ""
                                        },
                                        "title": "Interior Shady Heat Index (F)"
                                    },
                                    {
                                        "field": "Compare Heat Index (F)",
                                        "type": "quantitative",
                                        "formatType": "NUMBER_FORMATTER",
                                        "format": {
                                            "format": "NUMBER",
                                            "columnType": "NUMBER",
                                            "numDecimalDigits": -1,
                                            "currency": "$",
                                            "nanFormat": ""
                                        },
                                        "title": "Interior Sunny Heat Index (F)"
                                    },
                                    {
                                        "field": "Outdoor Heat Index (F)",
                                        "type": "quantitative",
                                        "formatType": "NUMBER_FORMATTER",
                                        "format": {
                                            "format": "NUMBER",
                                            "columnType": "NUMBER",
                                            "numDecimalDigits": -1,
                                            "currency": "$",
                                            "nanFormat": ""
                                        },
                                        "title": "Outdoor Heat Index (F)"
                                    },
                                    {
                                        "field": "A/C Is On",
                                        "type": "quantitative",
                                        "formatType": "NUMBER_FORMATTER",
                                        "format": {
                                            "format": "NUMBER",
                                            "columnType": "NUMBER",
                                            "numDecimalDigits": -1,
                                            "currency": "$",
                                            "nanFormat": ""
                                        },
                                        "title": "A/C Is On"
                                    }
                                ]
                            },
                            "params": [
                                {
                                    "name": "shared_pivot_hover_param",
                                    "select": {
                                        "type": "point",
                                        "fields": [
                                            "Hour (ET)"
                                        ],
                                        "nearest": true,
                                        "on": "mouseover",
                                        "clear": "mouseout"
                                    }
                                }
                            ]
                        }
                    ]
                }
            ],
            "transform": [
                {
                    "calculate": "toDate(datum[\"Hour (ET)\"])",
                    "as": "Hour (ET)"
                },
                {
                    "filter": "isValid(datum[\"Hour (ET)\"])"
                }
            ]
        }
    ],
    "config": {
        "legend": {
            "orient": "right"
        },
        "font": "\"IBM Plex Sans\", system-ui, -apple-system, BlinkMacSystemFont, sans-serif",
        "view": {}
    },
    "datasets": {
        "layer00": [
            {
                "name": "dummy",
                "value": 0
            }
        ]
    },
    "usermeta": {
        "selectionConfigs": {},
        "columnNameMappings": {}
    }
}
""")
chart_temp_hum_w_public_df.datasets.layer00 = temp_hum_w_public_df.to_json(orient='records')
chart_temp_hum_w_public_df.display(actions=False)

In [ ]:
import altair
chart_temp_hum_w_public_df = altair.Chart.from_json(r"""
{
    "width": "container",
    "height": "container",
    "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
    "layer": [
        {
            "description": "outer data layer",
            "resolve": {
                "scale": {
                    "color": "independent",
                    "y": "shared"
                }
            },
            "layer": [
                {
                    "description": "line series layer",
                    "transform": [],
                    "layer": [
                        {
                            "description": "line mark layer",
                            "mark": {
                                "type": "line",
                                "point": false,
                                "clip": true,
                                "strokeWidth": 3,
                                "strokeCap": "square",
                                "strokeJoin": "round",
                                "cursor": "crosshair"
                            },
                            "encoding": {
                                "opacity": {
                                    "value": 1
                                }
                            }
                        },
                        {
                            "description": "pivot tooltip point mark layer",
                            "mark": {
                                "type": "point",
                                "tooltip": false,
                                "clip": true
                            },
                            "encoding": {
                                "opacity": {
                                    "value": 0,
                                    "condition": {
                                        "param": "shared_pivot_hover_param",
                                        "value": 1,
                                        "empty": false
                                    }
                                },
                                "size": {
                                    "value": 80
                                }
                            }
                        }
                    ],
                    "encoding": {
                        "x": {
                            "field": "Hour (ET)",
                            "type": "temporal",
                            "title": "",
                            "scale": {},
                            "axis": {
                                "grid": true,
                                "ticks": true,
                                "labels": true,
                                "labelFlush": false
                            }
                        },
                        "y": {
                            "field": "Control Humidity (RH%)",
                            "type": "quantitative",
                            "title": "Relative Humidity (%)",
                            "scale": {},
                            "axis": {
                                "grid": true,
                                "ticks": true,
                                "labels": true,
                                "labelFlush": false,
                                "format": {
                                    "format": "NUMBER",
                                    "columnType": "NUMBER",
                                    "numDecimalDigits": -1,
                                    "currency": "$",
                                    "nanFormat": ""
                                },
                                "formatType": "NUMBER_FORMATTER"
                            }
                        },
                        "color": {
                            "datum": "Interior Shady Humidity (RH%)",
                            "scale": {
                                "range": [
                                    "#4C78A8",
                                    "#4C78A8"
                                ]
                            },
                            "legend": {
                                "symbolOpacity": 1,
                                "symbolType": "stroke"
                            }
                        }
                    }
                },
                {
                    "description": "line series layer",
                    "transform": [],
                    "layer": [
                        {
                            "description": "line mark layer",
                            "mark": {
                                "type": "line",
                                "point": false,
                                "clip": true,
                                "strokeWidth": 3,
                                "strokeCap": "square",
                                "strokeJoin": "round",
                                "cursor": "crosshair"
                            },
                            "encoding": {
                                "opacity": {
                                    "value": 1
                                }
                            }
                        },
                        {
                            "description": "pivot tooltip point mark layer",
                            "mark": {
                                "type": "point",
                                "tooltip": false,
                                "clip": true
                            },
                            "encoding": {
                                "opacity": {
                                    "value": 0,
                                    "condition": {
                                        "param": "shared_pivot_hover_param",
                                        "value": 1,
                                        "empty": false
                                    }
                                },
                                "size": {
                                    "value": 80
                                }
                            }
                        }
                    ],
                    "encoding": {
                        "x": {
                            "field": "Hour (ET)",
                            "type": "temporal",
                            "title": "",
                            "scale": {},
                            "axis": {
                                "grid": true,
                                "ticks": true,
                                "labels": true,
                                "labelFlush": false
                            }
                        },
                        "y": {
                            "field": "Compare Humidity (RH%)",
                            "type": "quantitative",
                            "title": "Relative Humidity (%)",
                            "scale": {},
                            "axis": {
                                "grid": true,
                                "ticks": true,
                                "labels": true,
                                "labelFlush": false,
                                "format": {
                                    "format": "NUMBER",
                                    "columnType": "NUMBER",
                                    "numDecimalDigits": -1,
                                    "currency": "$",
                                    "nanFormat": ""
                                },
                                "formatType": "NUMBER_FORMATTER"
                            }
                        },
                        "color": {
                            "datum": "Interior  Sunny Humidity (RH%)",
                            "scale": {
                                "range": [
                                    "#F58518",
                                    "#F58518"
                                ]
                            },
                            "legend": {
                                "symbolOpacity": 1,
                                "symbolType": "stroke"
                            }
                        }
                    }
                },
                {
                    "description": "line series layer",
                    "transform": [],
                    "layer": [
                        {
                            "description": "line mark layer",
                            "mark": {
                                "type": "line",
                                "point": false,
                                "clip": true,
                                "strokeWidth": 3,
                                "strokeCap": "square",
                                "strokeJoin": "round",
                                "cursor": "crosshair"
                            },
                            "encoding": {
                                "opacity": {
                                    "value": 1
                                }
                            }
                        },
                        {
                            "description": "pivot tooltip point mark layer",
                            "mark": {
                                "type": "point",
                                "tooltip": false,
                                "clip": true
                            },
                            "encoding": {
                                "opacity": {
                                    "value": 0,
                                    "condition": {
                                        "param": "shared_pivot_hover_param",
                                        "value": 1,
                                        "empty": false
                                    }
                                },
                                "size": {
                                    "value": 80
                                }
                            }
                        }
                    ],
                    "encoding": {
                        "x": {
                            "field": "Hour (ET)",
                            "type": "temporal",
                            "title": "",
                            "scale": {},
                            "axis": {
                                "grid": true,
                                "ticks": true,
                                "labels": true,
                                "labelFlush": false
                            }
                        },
                        "y": {
                            "field": "Outdoor Humidity (RH%)",
                            "type": "quantitative",
                            "title": "Relative Humidity (%)",
                            "scale": {},
                            "axis": {
                                "grid": true,
                                "ticks": true,
                                "labels": true,
                                "labelFlush": false,
                                "format": {
                                    "format": "NUMBER",
                                    "columnType": "NUMBER",
                                    "numDecimalDigits": -1,
                                    "currency": "$",
                                    "nanFormat": ""
                                },
                                "formatType": "NUMBER_FORMATTER"
                            }
                        },
                        "color": {
                            "datum": "Outdoor Humidity (RH%)",
                            "scale": {
                                "range": [
                                    "#E45756",
                                    "#E45756"
                                ]
                            },
                            "legend": {
                                "symbolOpacity": 1,
                                "symbolType": "stroke"
                            }
                        }
                    }
                },
                {
                    "description": "area series layer",
                    "transform": [],
                    "layer": [
                        {
                            "description": "background area mark layer",
                            "mark": {
                                "type": "area",
                                "point": false,
                                "line": true,
                                "strokeJoin": "round",
                                "clip": true,
                                "cursor": "crosshair"
                            },
                            "encoding": {
                                "color": {
                                    "datum": "A/C Is On",
                                    "scale": {
                                        "range": [
                                            "#4C78A8",
                                            "#4C78A8"
                                        ]
                                    },
                                    "legend": {
                                        "symbolOpacity": 0.15
                                    }
                                },
                                "opacity": {
                                    "value": 0.2
                                }
                            }
                        },
                        {
                            "transform": [],
                            "description": "area mark layer",
                            "mark": {
                                "type": "area",
                                "point": false,
                                "line": true,
                                "clip": true
                            },
                            "encoding": {
                                "color": {
                                    "datum": "A/C Is On",
                                    "scale": {
                                        "range": [
                                            "#4C78A8",
                                            "#4C78A8"
                                        ]
                                    },
                                    "legend": {
                                        "symbolOpacity": 0.15
                                    }
                                },
                                "opacity": {
                                    "value": 0.15
                                }
                            }
                        },
                        {
                            "description": "pivot tooltip point mark layer",
                            "mark": {
                                "type": "point",
                                "tooltip": false,
                                "clip": true
                            },
                            "encoding": {
                                "opacity": {
                                    "value": 0,
                                    "condition": {
                                        "param": "shared_pivot_hover_param",
                                        "value": 1,
                                        "empty": false
                                    }
                                },
                                "size": {
                                    "value": 80
                                },
                                "color": {
                                    "datum": "A/C Is On",
                                    "scale": {
                                        "range": [
                                            "#4C78A8",
                                            "#4C78A8"
                                        ]
                                    },
                                    "legend": {
                                        "symbolOpacity": 0.15
                                    }
                                },
                                "y": {
                                    "field": "A/C Is On",
                                    "type": "quantitative",
                                    "title": "Relative Humidity (%)",
                                    "scale": {},
                                    "axis": {
                                        "grid": true,
                                        "ticks": true,
                                        "labels": true,
                                        "labelFlush": false,
                                        "format": {
                                            "format": "NUMBER",
                                            "columnType": "NUMBER",
                                            "numDecimalDigits": -1,
                                            "currency": "$",
                                            "nanFormat": ""
                                        },
                                        "formatType": "NUMBER_FORMATTER"
                                    },
                                    "stack": "zero"
                                }
                            }
                        }
                    ],
                    "encoding": {
                        "x": {
                            "field": "Hour (ET)",
                            "type": "temporal",
                            "title": "",
                            "scale": {},
                            "axis": {
                                "grid": true,
                                "ticks": true,
                                "labels": true,
                                "labelFlush": false
                            }
                        },
                        "y": {
                            "field": "A/C Is On",
                            "type": "quantitative",
                            "title": "Relative Humidity (%)",
                            "scale": {},
                            "axis": {
                                "grid": true,
                                "ticks": true,
                                "labels": true,
                                "labelFlush": false,
                                "format": {
                                    "format": "NUMBER",
                                    "columnType": "NUMBER",
                                    "numDecimalDigits": -1,
                                    "currency": "$",
                                    "nanFormat": ""
                                },
                                "formatType": "NUMBER_FORMATTER"
                            }
                        }
                    }
                },
                {
                    "description": "shared tooltip layer",
                    "encoding": {
                        "x": {
                            "field": "Hour (ET)",
                            "type": "temporal",
                            "title": "",
                            "scale": {},
                            "axis": {
                                "grid": true,
                                "ticks": true,
                                "labels": true,
                                "labelFlush": false
                            }
                        }
                    },
                    "layer": [
                        {
                            "description": "shared tooltip rule mark layer",
                            "mark": {
                                "type": "rule",
                                "clip": true
                            },
                            "encoding": {
                                "opacity": {
                                    "condition": {
                                        "value": 0.3,
                                        "param": "shared_pivot_hover_param",
                                        "empty": false
                                    },
                                    "value": 0
                                },
                                "color": {
                                    "value": "CHART_DEFAULT_RULE_COLOR_MARKER"
                                },
                                "tooltip": [
                                    {
                                        "field": "Hour (ET)",
                                        "type": "temporal",
                                        "title": " "
                                    },
                                    {
                                        "field": "Control Humidity (RH%)",
                                        "type": "quantitative",
                                        "formatType": "NUMBER_FORMATTER",
                                        "format": {
                                            "format": "NUMBER",
                                            "columnType": "NUMBER",
                                            "numDecimalDigits": -1,
                                            "currency": "$",
                                            "nanFormat": ""
                                        },
                                        "title": "Interior Shady Humidity (RH%)"
                                    },
                                    {
                                        "field": "Compare Humidity (RH%)",
                                        "type": "quantitative",
                                        "formatType": "NUMBER_FORMATTER",
                                        "format": {
                                            "format": "NUMBER",
                                            "columnType": "NUMBER",
                                            "numDecimalDigits": -1,
                                            "currency": "$",
                                            "nanFormat": ""
                                        },
                                        "title": "Interior  Sunny Humidity (RH%)"
                                    },
                                    {
                                        "field": "Outdoor Humidity (RH%)",
                                        "type": "quantitative",
                                        "formatType": "NUMBER_FORMATTER",
                                        "format": {
                                            "format": "NUMBER",
                                            "columnType": "NUMBER",
                                            "numDecimalDigits": -1,
                                            "currency": "$",
                                            "nanFormat": ""
                                        },
                                        "title": "Outdoor Humidity (RH%)"
                                    },
                                    {
                                        "field": "A/C Is On",
                                        "type": "quantitative",
                                        "formatType": "NUMBER_FORMATTER",
                                        "format": {
                                            "format": "NUMBER",
                                            "columnType": "NUMBER",
                                            "numDecimalDigits": -1,
                                            "currency": "$",
                                            "nanFormat": ""
                                        },
                                        "title": "A/C Is On"
                                    }
                                ]
                            },
                            "params": [
                                {
                                    "name": "shared_pivot_hover_param",
                                    "select": {
                                        "type": "point",
                                        "fields": [
                                            "Hour (ET)"
                                        ],
                                        "nearest": true,
                                        "on": "mouseover",
                                        "clear": "mouseout"
                                    }
                                }
                            ]
                        }
                    ]
                }
            ],
            "transform": [
                {
                    "calculate": "toDate(datum[\"Hour (ET)\"])",
                    "as": "Hour (ET)"
                },
                {
                    "filter": "isValid(datum[\"Hour (ET)\"])"
                }
            ]
        }
    ],
    "config": {
        "legend": {
            "orient": "right"
        },
        "font": "\"IBM Plex Sans\", system-ui, -apple-system, BlinkMacSystemFont, sans-serif",
        "view": {}
    },
    "datasets": {
        "layer00": [
            {
                "name": "dummy",
                "value": 0
            }
        ]
    },
    "usermeta": {
        "selectionConfigs": {},
        "columnNameMappings": {}
    }
}
""")
chart_temp_hum_w_public_df.datasets.layer00 = temp_hum_w_public_df.to_json(orient='records')
chart_temp_hum_w_public_df.display(actions=False)

In [ ]:
# Calculate the differences in temperature between Control vs Compare and Control vs Outdoors
compare_temp_HI_df = temp_hum_w_public_df[['Hour (ET)', 'Control Temp (F)', 'Compare Temp (F)', 'Outdoor Temp (F)','Control Heat Index (F)', 'Compare Heat Index (F)', 'Outdoor Heat Index (F)']].copy()
compare_temp_HI_df.loc[:, 'Temp: Compare - Control'] = (compare_temp_HI_df['Compare Temp (F)'] - compare_temp_HI_df['Control Temp (F)']).round(2)
compare_temp_HI_df.loc[:, 'Temp: Outdoor - Control'] = (compare_temp_HI_df['Outdoor Temp (F)'] - compare_temp_HI_df['Control Temp (F)']).round(2)
compare_temp_HI_df.loc[:, 'Temp: Outdoor - Compare'] = (compare_temp_HI_df['Outdoor Temp (F)'] - compare_temp_HI_df['Compare Temp (F)']).round(2)
compare_temp_HI_df.loc[:, 'HI: Compare - Control'] = (compare_temp_HI_df['Compare Heat Index (F)'] - compare_temp_HI_df['Control Heat Index (F)']).round(2)
compare_temp_HI_df.loc[:, 'HI: Outdoor - Control'] = (compare_temp_HI_df['Outdoor Heat Index (F)'] - compare_temp_HI_df['Control Heat Index (F)']).round(2)
compare_temp_HI_df.loc[:, 'HI: Outdoor - Compare'] = (compare_temp_HI_df['Outdoor Heat Index (F)'] - compare_temp_HI_df['Compare Heat Index (F)']).round(2)
compare_temp_HI_df.loc[:, 'Baseline'] = 0

compare_temp_HI_df

,Hour (ET),Control Temp (F),Compare Temp (F),Outdoor Temp (F),Control Heat Index (F),Compare Heat Index (F),Outdoor Heat Index (F),Temp: Compare - Control,Temp: Outdoor - Control,Temp: Outdoor - Compare,HI: Compare - Control,HI: Outdoor - Control,HI: Outdoor - Compare,Baseline
0,2024-08-12 19:00:00,80.03498,80.60288,79.4,82.2,82.6,79.0,0.57,-0.63,-1.20,0.4,-3.2,-3.6,0
1,2024-08-12 18:00:00,80.07188,81.80798,80.5,82.3,84.2,80.8,1.74,0.43,-1.31,1.9,-1.5,-3.4,0
2,2024-08-12 17:00:00,79.98890,82.32728,78.0,82.1,84.7,78.2,2.34,-1.99,-4.33,2.6,-3.9,-6.5,0
3,2024-08-12 16:00:00,80.11796,82.10012,80.5,82.4,84.6,81.2,1.98,0.38,-1.60,2.2,-1.2,-3.4,0
4,2024-08-12 15:00:00,79.94750,83.62526,82.6,82.2,86.4,82.4,3.68,2.65,-1.03,4.2,0.2,-4.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
426,2024-07-25 21:00:00,74.26976,75.85574,79.2,74.5,76.1,79.1,1.59,4.93,3.34,1.6,4.6,3.0,0
427,2024-07-25 20:00:00,80.23784,80.87180,81.1,82.9,83.6,81.5,0.63,0.86,0.23,0.7,-1.4,-2.1,0
428,2024-07-25 19:00:00,80.26574,81.44636,82.1,83.0,84.4,82.6,1.18,1.83,0.65,1.4,-0.4,-1.8,0
429,2024-07-25 18:00:00,80.33954,82.16024,82.2,83.4,85.9,83.3,1.82,1.86,0.04,2.5,-0.1,-2.6,0


In [ ]:
import altair
chart_compare_temp_HI_df = altair.Chart.from_json(r"""
{
    "width": "container",
    "height": "container",
    "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
    "layer": [
        {
            "description": "outer data layer",
            "resolve": {
                "scale": {
                    "color": "independent",
                    "y": "shared"
                }
            },
            "layer": [
                {
                    "description": "line series layer",
                    "transform": [],
                    "layer": [
                        {
                            "description": "line mark layer",
                            "mark": {
                                "type": "line",
                                "point": false,
                                "clip": true,
                                "strokeWidth": 5,
                                "strokeCap": "square",
                                "strokeJoin": "round",
                                "cursor": "crosshair"
                            },
                            "encoding": {
                                "opacity": {
                                    "value": 1
                                }
                            }
                        },
                        {
                            "description": "pivot tooltip point mark layer",
                            "mark": {
                                "type": "point",
                                "tooltip": false,
                                "clip": true
                            },
                            "encoding": {
                                "opacity": {
                                    "value": 0,
                                    "condition": {
                                        "param": "shared_pivot_hover_param",
                                        "value": 1,
                                        "empty": false
                                    }
                                },
                                "size": {
                                    "value": 80
                                }
                            }
                        }
                    ],
                    "encoding": {
                        "x": {
                            "field": "Hour (ET)",
                            "type": "temporal",
                            "title": "",
                            "scale": {},
                            "axis": {
                                "grid": true,
                                "ticks": true,
                                "labels": true,
                                "labelFlush": false
                            }
                        },
                        "y": {
                            "field": "Baseline",
                            "type": "quantitative",
                            "title": "Degrees Difference (F)",
                            "scale": {},
                            "axis": {
                                "grid": true,
                                "ticks": true,
                                "labels": true,
                                "labelFlush": false,
                                "format": {
                                    "format": "NUMBER",
                                    "columnType": "NUMBER",
                                    "numDecimalDigits": -1,
                                    "currency": "$",
                                    "nanFormat": ""
                                },
                                "formatType": "NUMBER_FORMATTER"
                            }
                        },
                        "color": {
                            "datum": "Baseline",
                            "scale": {
                                "range": [
                                    "#4C78A8",
                                    "#4C78A8"
                                ]
                            },
                            "legend": {
                                "symbolOpacity": 1,
                                "symbolType": "stroke"
                            }
                        }
                    }
                },
                {
                    "description": "area series layer",
                    "transform": [],
                    "layer": [
                        {
                            "description": "background area mark layer",
                            "mark": {
                                "type": "area",
                                "point": false,
                                "line": false,
                                "strokeJoin": "round",
                                "clip": true,
                                "cursor": "crosshair"
                            },
                            "encoding": {
                                "color": {
                                    "datum": "Temp: Compare - Control",
                                    "scale": {
                                        "range": [
                                            "#F58518",
                                            "#F58518"
                                        ]
                                    },
                                    "legend": {
                                        "symbolOpacity": 0.8
                                    }
                                },
                                "opacity": {
                                    "value": 0.2
                                }
                            }
                        },
                        {
                            "transform": [],
                            "description": "area mark layer",
                            "mark": {
                                "type": "area",
                                "point": false,
                                "line": false,
                                "clip": true
                            },
                            "encoding": {
                                "color": {
                                    "datum": "Temp: Compare - Control",
                                    "scale": {
                                        "range": [
                                            "#F58518",
                                            "#F58518"
                                        ]
                                    },
                                    "legend": {
                                        "symbolOpacity": 0.8
                                    }
                                },
                                "opacity": {
                                    "value": 0.8
                                }
                            }
                        },
                        {
                            "description": "pivot tooltip point mark layer",
                            "mark": {
                                "type": "point",
                                "tooltip": false,
                                "clip": true
                            },
                            "encoding": {
                                "opacity": {
                                    "value": 0,
                                    "condition": {
                                        "param": "shared_pivot_hover_param",
                                        "value": 1,
                                        "empty": false
                                    }
                                },
                                "size": {
                                    "value": 80
                                },
                                "color": {
                                    "datum": "Temp: Compare - Control",
                                    "scale": {
                                        "range": [
                                            "#F58518",
                                            "#F58518"
                                        ]
                                    },
                                    "legend": {
                                        "symbolOpacity": 0.8
                                    }
                                },
                                "y": {
                                    "field": "Temp: Compare - Control",
                                    "type": "quantitative",
                                    "title": "Degrees Difference (F)",
                                    "scale": {},
                                    "axis": {
                                        "grid": true,
                                        "ticks": true,
                                        "labels": true,
                                        "labelFlush": false,
                                        "format": {
                                            "format": "NUMBER",
                                            "columnType": "NUMBER",
                                            "numDecimalDigits": -1,
                                            "currency": "$",
                                            "nanFormat": ""
                                        },
                                        "formatType": "NUMBER_FORMATTER"
                                    },
                                    "stack": "zero"
                                }
                            }
                        }
                    ],
                    "encoding": {
                        "x": {
                            "field": "Hour (ET)",
                            "type": "temporal",
                            "title": "",
                            "scale": {},
                            "axis": {
                                "grid": true,
                                "ticks": true,
                                "labels": true,
                                "labelFlush": false
                            }
                        },
                        "y": {
                            "field": "Temp: Compare - Control",
                            "type": "quantitative",
                            "title": "Degrees Difference (F)",
                            "scale": {},
                            "axis": {
                                "grid": true,
                                "ticks": true,
                                "labels": true,
                                "labelFlush": false,
                                "format": {
                                    "format": "NUMBER",
                                    "columnType": "NUMBER",
                                    "numDecimalDigits": -1,
                                    "currency": "$",
                                    "nanFormat": ""
                                },
                                "formatType": "NUMBER_FORMATTER"
                            }
                        }
                    }
                },
                {
                    "description": "area series layer",
                    "transform": [],
                    "layer": [
                        {
                            "description": "background area mark layer",
                            "mark": {
                                "type": "area",
                                "point": false,
                                "line": false,
                                "strokeJoin": "round",
                                "clip": true,
                                "cursor": "crosshair"
                            },
                            "encoding": {
                                "color": {
                                    "datum": "Temp: Outdoor - Control",
                                    "scale": {
                                        "range": [
                                            "#E45756",
                                            "#E45756"
                                        ]
                                    },
                                    "legend": {
                                        "symbolOpacity": 0.5
                                    }
                                },
                                "opacity": {
                                    "value": 0.2
                                }
                            }
                        },
                        {
                            "transform": [],
                            "description": "area mark layer",
                            "mark": {
                                "type": "area",
                                "point": false,
                                "line": false,
                                "clip": true
                            },
                            "encoding": {
                                "color": {
                                    "datum": "Temp: Outdoor - Control",
                                    "scale": {
                                        "range": [
                                            "#E45756",
                                            "#E45756"
                                        ]
                                    },
                                    "legend": {
                                        "symbolOpacity": 0.5
                                    }
                                },
                                "opacity": {
                                    "value": 0.5
                                }
                            }
                        },
                        {
                            "description": "pivot tooltip point mark layer",
                            "mark": {
                                "type": "point",
                                "tooltip": false,
                                "clip": true
                            },
                            "encoding": {
                                "opacity": {
                                    "value": 0,
                                    "condition": {
                                        "param": "shared_pivot_hover_param",
                                        "value": 1,
                                        "empty": false
                                    }
                                },
                                "size": {
                                    "value": 80
                                },
                                "color": {
                                    "datum": "Temp: Outdoor - Control",
                                    "scale": {
                                        "range": [
                                            "#E45756",
                                            "#E45756"
                                        ]
                                    },
                                    "legend": {
                                        "symbolOpacity": 0.5
                                    }
                                },
                                "y": {
                                    "field": "Temp: Outdoor - Control",
                                    "type": "quantitative",
                                    "title": "Degrees Difference (F)",
                                    "scale": {},
                                    "axis": {
                                        "grid": true,
                                        "ticks": true,
                                        "labels": true,
                                        "labelFlush": false,
                                        "format": {
                                            "format": "NUMBER",
                                            "columnType": "NUMBER",
                                            "numDecimalDigits": -1,
                                            "currency": "$",
                                            "nanFormat": ""
                                        },
                                        "formatType": "NUMBER_FORMATTER"
                                    },
                                    "stack": "zero"
                                }
                            }
                        }
                    ],
                    "encoding": {
                        "x": {
                            "field": "Hour (ET)",
                            "type": "temporal",
                            "title": "",
                            "scale": {},
                            "axis": {
                                "grid": true,
                                "ticks": true,
                                "labels": true,
                                "labelFlush": false
                            }
                        },
                        "y": {
                            "field": "Temp: Outdoor - Control",
                            "type": "quantitative",
                            "title": "Degrees Difference (F)",
                            "scale": {},
                            "axis": {
                                "grid": true,
                                "ticks": true,
                                "labels": true,
                                "labelFlush": false,
                                "format": {
                                    "format": "NUMBER",
                                    "columnType": "NUMBER",
                                    "numDecimalDigits": -1,
                                    "currency": "$",
                                    "nanFormat": ""
                                },
                                "formatType": "NUMBER_FORMATTER"
                            }
                        }
                    }
                },
                {
                    "description": "line series layer",
                    "transform": [],
                    "layer": [
                        {
                            "description": "line mark layer",
                            "mark": {
                                "type": "line",
                                "point": false,
                                "clip": true,
                                "strokeDash": [
                                    6,
                                    3
                                ],
                                "strokeWidth": 1,
                                "strokeJoin": "round",
                                "cursor": "crosshair"
                            },
                            "encoding": {
                                "opacity": {
                                    "value": 1
                                }
                            }
                        },
                        {
                            "description": "pivot tooltip point mark layer",
                            "mark": {
                                "type": "point",
                                "tooltip": false,
                                "clip": true
                            },
                            "encoding": {
                                "opacity": {
                                    "value": 0,
                                    "condition": {
                                        "param": "shared_pivot_hover_param",
                                        "value": 1,
                                        "empty": false
                                    }
                                },
                                "size": {
                                    "value": 80
                                }
                            }
                        }
                    ],
                    "encoding": {
                        "x": {
                            "field": "Hour (ET)",
                            "type": "temporal",
                            "title": "",
                            "scale": {},
                            "axis": {
                                "grid": true,
                                "ticks": true,
                                "labels": true,
                                "labelFlush": false
                            }
                        },
                        "y": {
                            "field": "HI: Compare - Control",
                            "type": "quantitative",
                            "title": "Degrees Difference (F)",
                            "scale": {},
                            "axis": {
                                "grid": true,
                                "ticks": true,
                                "labels": true,
                                "labelFlush": false,
                                "format": {
                                    "format": "NUMBER",
                                    "columnType": "NUMBER",
                                    "numDecimalDigits": -1,
                                    "currency": "$",
                                    "nanFormat": ""
                                },
                                "formatType": "NUMBER_FORMATTER"
                            }
                        },
                        "color": {
                            "datum": "Heat Index: Compare - Control",
                            "scale": {
                                "range": [
                                    "#F58518",
                                    "#F58518"
                                ]
                            },
                            "legend": {
                                "symbolOpacity": 1,
                                "symbolType": "stroke",
                                "symbolDash": [
                                    6,
                                    3
                                ],
                                "symbolDashOffset": 2.5
                            }
                        }
                    }
                },
                {
                    "description": "line series layer",
                    "transform": [],
                    "layer": [
                        {
                            "description": "line mark layer",
                            "mark": {
                                "type": "line",
                                "point": false,
                                "clip": true,
                                "strokeDash": [
                                    6,
                                    3
                                ],
                                "strokeWidth": 1,
                                "strokeJoin": "round",
                                "cursor": "crosshair"
                            },
                            "encoding": {
                                "opacity": {
                                    "value": 1
                                }
                            }
                        },
                        {
                            "description": "pivot tooltip point mark layer",
                            "mark": {
                                "type": "point",
                                "tooltip": false,
                                "clip": true
                            },
                            "encoding": {
                                "opacity": {
                                    "value": 0,
                                    "condition": {
                                        "param": "shared_pivot_hover_param",
                                        "value": 1,
                                        "empty": false
                                    }
                                },
                                "size": {
                                    "value": 80
                                }
                            }
                        }
                    ],
                    "encoding": {
                        "x": {
                            "field": "Hour (ET)",
                            "type": "temporal",
                            "title": "",
                            "scale": {},
                            "axis": {
                                "grid": true,
                                "ticks": true,
                                "labels": true,
                                "labelFlush": false
                            }
                        },
                        "y": {
                            "field": "HI: Outdoor - Control",
                            "type": "quantitative",
                            "title": "Degrees Difference (F)",
                            "scale": {},
                            "axis": {
                                "grid": true,
                                "ticks": true,
                                "labels": true,
                                "labelFlush": false,
                                "format": {
                                    "format": "NUMBER",
                                    "columnType": "NUMBER",
                                    "numDecimalDigits": -1,
                                    "currency": "$",
                                    "nanFormat": ""
                                },
                                "formatType": "NUMBER_FORMATTER"
                            }
                        },
                        "color": {
                            "datum": "Heat Index: Outdoor - Control",
                            "scale": {
                                "range": [
                                    "#E45756",
                                    "#E45756"
                                ]
                            },
                            "legend": {
                                "symbolOpacity": 1,
                                "symbolType": "stroke",
                                "symbolDash": [
                                    6,
                                    3
                                ],
                                "symbolDashOffset": 2.5
                            }
                        }
                    }
                },
                {
                    "description": "shared tooltip layer",
                    "encoding": {
                        "x": {
                            "field": "Hour (ET)",
                            "type": "temporal",
                            "title": "",
                            "scale": {},
                            "axis": {
                                "grid": true,
                                "ticks": true,
                                "labels": true,
                                "labelFlush": false
                            }
                        }
                    },
                    "layer": [
                        {
                            "description": "shared tooltip rule mark layer",
                            "mark": {
                                "type": "rule",
                                "clip": true
                            },
                            "encoding": {
                                "opacity": {
                                    "condition": {
                                        "value": 0.3,
                                        "param": "shared_pivot_hover_param",
                                        "empty": false
                                    },
                                    "value": 0
                                },
                                "color": {
                                    "value": "CHART_DEFAULT_RULE_COLOR_MARKER"
                                },
                                "tooltip": [
                                    {
                                        "field": "Hour (ET)",
                                        "type": "temporal",
                                        "title": " "
                                    },
                                    {
                                        "field": "Baseline",
                                        "type": "quantitative",
                                        "formatType": "NUMBER_FORMATTER",
                                        "format": {
                                            "format": "NUMBER",
                                            "columnType": "NUMBER",
                                            "numDecimalDigits": -1,
                                            "currency": "$",
                                            "nanFormat": ""
                                        },
                                        "title": "Baseline"
                                    },
                                    {
                                        "field": "Temp: Compare - Control",
                                        "type": "quantitative",
                                        "formatType": "NUMBER_FORMATTER",
                                        "format": {
                                            "format": "NUMBER",
                                            "columnType": "NUMBER",
                                            "numDecimalDigits": -1,
                                            "currency": "$",
                                            "nanFormat": ""
                                        },
                                        "title": "Temp: Compare - Control"
                                    },
                                    {
                                        "field": "Temp: Outdoor - Control",
                                        "type": "quantitative",
                                        "formatType": "NUMBER_FORMATTER",
                                        "format": {
                                            "format": "NUMBER",
                                            "columnType": "NUMBER",
                                            "numDecimalDigits": -1,
                                            "currency": "$",
                                            "nanFormat": ""
                                        },
                                        "title": "Temp: Outdoor - Control"
                                    },
                                    {
                                        "field": "HI: Compare - Control",
                                        "type": "quantitative",
                                        "formatType": "NUMBER_FORMATTER",
                                        "format": {
                                            "format": "NUMBER",
                                            "columnType": "NUMBER",
                                            "numDecimalDigits": -1,
                                            "currency": "$",
                                            "nanFormat": ""
                                        },
                                        "title": "Heat Index: Compare - Control"
                                    },
                                    {
                                        "field": "HI: Outdoor - Control",
                                        "type": "quantitative",
                                        "formatType": "NUMBER_FORMATTER",
                                        "format": {
                                            "format": "NUMBER",
                                            "columnType": "NUMBER",
                                            "numDecimalDigits": -1,
                                            "currency": "$",
                                            "nanFormat": ""
                                        },
                                        "title": "Heat Index: Outdoor - Control"
                                    }
                                ]
                            },
                            "params": [
                                {
                                    "name": "shared_pivot_hover_param",
                                    "select": {
                                        "type": "point",
                                        "fields": [
                                            "Hour (ET)"
                                        ],
                                        "nearest": true,
                                        "on": "mouseover",
                                        "clear": "mouseout"
                                    }
                                }
                            ]
                        }
                    ]
                }
            ],
            "transform": [
                {
                    "calculate": "toDate(datum[\"Hour (ET)\"])",
                    "as": "Hour (ET)"
                },
                {
                    "filter": "isValid(datum[\"Hour (ET)\"])"
                }
            ]
        }
    ],
    "config": {
        "legend": {
            "orient": "right"
        },
        "font": "\"IBM Plex Sans\", system-ui, -apple-system, BlinkMacSystemFont, sans-serif",
        "view": {}
    },
    "datasets": {
        "layer00": [
            {
                "name": "dummy",
                "value": 0
            }
        ]
    },
    "usermeta": {
        "selectionConfigs": {},
        "columnNameMappings": {}
    }
}
""")
chart_compare_temp_HI_df.datasets.layer00 = compare_temp_HI_df.to_json(orient='records')
chart_compare_temp_HI_df.display(actions=False)

#### It's All Relative
Examining the differences of the Interior versus Outdoor temperatures, and Interior versus Interior Sunny temperatures sheds a light on the power of simply closing blinds.

In [ ]:
import altair
chart_compare_temp_HI_df = altair.Chart.from_json(r"""
{
    "width": "container",
    "height": "container",
    "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
    "layer": [
        {
            "description": "outer data layer",
            "resolve": {
                "scale": {
                    "color": "independent",
                    "y": "shared"
                }
            },
            "layer": [
                {
                    "description": "line series layer",
                    "transform": [],
                    "layer": [
                        {
                            "description": "line mark layer",
                            "mark": {
                                "type": "line",
                                "point": false,
                                "clip": true,
                                "strokeWidth": 5,
                                "strokeCap": "square",
                                "strokeJoin": "round",
                                "cursor": "crosshair"
                            },
                            "encoding": {
                                "opacity": {
                                    "value": 1
                                }
                            }
                        },
                        {
                            "description": "pivot tooltip point mark layer",
                            "mark": {
                                "type": "point",
                                "tooltip": false,
                                "clip": true
                            },
                            "encoding": {
                                "opacity": {
                                    "value": 0,
                                    "condition": {
                                        "param": "shared_pivot_hover_param",
                                        "value": 1,
                                        "empty": false
                                    }
                                },
                                "size": {
                                    "value": 80
                                }
                            }
                        }
                    ],
                    "encoding": {
                        "x": {
                            "field": "Hour (ET)",
                            "type": "temporal",
                            "title": "",
                            "scale": {},
                            "axis": {
                                "grid": true,
                                "ticks": true,
                                "labels": true,
                                "labelFlush": false
                            }
                        },
                        "y": {
                            "field": "Baseline",
                            "type": "quantitative",
                            "aggregate": "sum",
                            "title": "Degrees Difference (F)",
                            "scale": {
                                "domainMin": -14,
                                "domainMax": 24
                            },
                            "axis": {
                                "grid": true,
                                "ticks": true,
                                "labels": true,
                                "labelFlush": false,
                                "format": {
                                    "format": "NUMBER",
                                    "columnType": "NUMBER",
                                    "numDecimalDigits": -1,
                                    "currency": "$",
                                    "nanFormat": ""
                                },
                                "formatType": "NUMBER_FORMATTER"
                            }
                        },
                        "color": {
                            "datum": "Interior / Baseline",
                            "scale": {
                                "range": [
                                    "#BAB0AC",
                                    "#BAB0AC"
                                ]
                            },
                            "legend": {
                                "symbolOpacity": 1,
                                "symbolType": "stroke"
                            }
                        }
                    }
                },
                {
                    "description": "area series layer",
                    "transform": [],
                    "layer": [
                        {
                            "description": "background area mark layer",
                            "mark": {
                                "type": "area",
                                "point": false,
                                "line": true,
                                "strokeJoin": "round",
                                "clip": true,
                                "cursor": "crosshair"
                            },
                            "encoding": {
                                "color": {
                                    "datum": "Difference Between Interior Shady and Outdoor",
                                    "scale": {
                                        "range": [
                                            "#E45756",
                                            "#E45756"
                                        ]
                                    },
                                    "legend": {
                                        "symbolOpacity": 1
                                    }
                                },
                                "opacity": {
                                    "value": 0.2
                                }
                            }
                        },
                        {
                            "transform": [],
                            "description": "area mark layer",
                            "mark": {
                                "type": "area",
                                "point": false,
                                "line": true,
                                "clip": true
                            },
                            "encoding": {
                                "color": {
                                    "datum": "Difference Between Interior Shady and Outdoor",
                                    "scale": {
                                        "range": [
                                            "#E45756",
                                            "#E45756"
                                        ]
                                    },
                                    "legend": {
                                        "symbolOpacity": 1
                                    }
                                }
                            }
                        },
                        {
                            "description": "pivot tooltip point mark layer",
                            "mark": {
                                "type": "point",
                                "tooltip": false,
                                "clip": true
                            },
                            "encoding": {
                                "opacity": {
                                    "value": 0,
                                    "condition": {
                                        "param": "shared_pivot_hover_param",
                                        "value": 1,
                                        "empty": false
                                    }
                                },
                                "size": {
                                    "value": 80
                                },
                                "color": {
                                    "datum": "Difference Between Interior Shady and Outdoor",
                                    "scale": {
                                        "range": [
                                            "#E45756",
                                            "#E45756"
                                        ]
                                    },
                                    "legend": {
                                        "symbolOpacity": 1
                                    }
                                },
                                "y": {
                                    "field": "Temp: Outdoor - Control",
                                    "type": "quantitative",
                                    "title": "Degrees Difference (F)",
                                    "scale": {
                                        "domainMin": -14,
                                        "domainMax": 24
                                    },
                                    "axis": {
                                        "grid": true,
                                        "ticks": true,
                                        "labels": true,
                                        "labelFlush": false,
                                        "format": {
                                            "format": "NUMBER",
                                            "columnType": "NUMBER",
                                            "numDecimalDigits": -1,
                                            "currency": "$",
                                            "nanFormat": ""
                                        },
                                        "formatType": "NUMBER_FORMATTER"
                                    },
                                    "stack": "zero"
                                }
                            }
                        }
                    ],
                    "encoding": {
                        "x": {
                            "field": "Hour (ET)",
                            "type": "temporal",
                            "title": "",
                            "scale": {},
                            "axis": {
                                "grid": true,
                                "ticks": true,
                                "labels": true,
                                "labelFlush": false
                            }
                        },
                        "y": {
                            "field": "Temp: Outdoor - Control",
                            "type": "quantitative",
                            "title": "Degrees Difference (F)",
                            "scale": {
                                "domainMin": -14,
                                "domainMax": 24
                            },
                            "axis": {
                                "grid": true,
                                "ticks": true,
                                "labels": true,
                                "labelFlush": false,
                                "format": {
                                    "format": "NUMBER",
                                    "columnType": "NUMBER",
                                    "numDecimalDigits": -1,
                                    "currency": "$",
                                    "nanFormat": ""
                                },
                                "formatType": "NUMBER_FORMATTER"
                            }
                        }
                    }
                },
                {
                    "description": "area series layer",
                    "transform": [],
                    "layer": [
                        {
                            "description": "background area mark layer",
                            "mark": {
                                "type": "area",
                                "point": false,
                                "line": true,
                                "strokeJoin": "round",
                                "clip": true,
                                "cursor": "crosshair"
                            },
                            "encoding": {
                                "color": {
                                    "datum": "Difference Between Interior Shady and Intrerior Sunny",
                                    "scale": {
                                        "range": [
                                            "#F58518",
                                            "#F58518"
                                        ]
                                    },
                                    "legend": {
                                        "symbolOpacity": 1
                                    }
                                },
                                "opacity": {
                                    "value": 0.2
                                }
                            }
                        },
                        {
                            "transform": [],
                            "description": "area mark layer",
                            "mark": {
                                "type": "area",
                                "point": false,
                                "line": true,
                                "clip": true
                            },
                            "encoding": {
                                "color": {
                                    "datum": "Difference Between Interior Shady and Intrerior Sunny",
                                    "scale": {
                                        "range": [
                                            "#F58518",
                                            "#F58518"
                                        ]
                                    },
                                    "legend": {
                                        "symbolOpacity": 1
                                    }
                                }
                            }
                        },
                        {
                            "description": "pivot tooltip point mark layer",
                            "mark": {
                                "type": "point",
                                "tooltip": false,
                                "clip": true
                            },
                            "encoding": {
                                "opacity": {
                                    "value": 0,
                                    "condition": {
                                        "param": "shared_pivot_hover_param",
                                        "value": 1,
                                        "empty": false
                                    }
                                },
                                "size": {
                                    "value": 80
                                },
                                "color": {
                                    "datum": "Difference Between Interior Shady and Intrerior Sunny",
                                    "scale": {
                                        "range": [
                                            "#F58518",
                                            "#F58518"
                                        ]
                                    },
                                    "legend": {
                                        "symbolOpacity": 1
                                    }
                                },
                                "y": {
                                    "field": "Temp: Compare - Control",
                                    "type": "quantitative",
                                    "title": "Degrees Difference (F)",
                                    "scale": {
                                        "domainMin": -14,
                                        "domainMax": 24
                                    },
                                    "axis": {
                                        "grid": true,
                                        "ticks": true,
                                        "labels": true,
                                        "labelFlush": false,
                                        "format": {
                                            "format": "NUMBER",
                                            "columnType": "NUMBER",
                                            "numDecimalDigits": -1,
                                            "currency": "$",
                                            "nanFormat": ""
                                        },
                                        "formatType": "NUMBER_FORMATTER"
                                    },
                                    "stack": "zero"
                                }
                            }
                        }
                    ],
                    "encoding": {
                        "x": {
                            "field": "Hour (ET)",
                            "type": "temporal",
                            "title": "",
                            "scale": {},
                            "axis": {
                                "grid": true,
                                "ticks": true,
                                "labels": true,
                                "labelFlush": false
                            }
                        },
                        "y": {
                            "field": "Temp: Compare - Control",
                            "type": "quantitative",
                            "title": "Degrees Difference (F)",
                            "scale": {
                                "domainMin": -14,
                                "domainMax": 24
                            },
                            "axis": {
                                "grid": true,
                                "ticks": true,
                                "labels": true,
                                "labelFlush": false,
                                "format": {
                                    "format": "NUMBER",
                                    "columnType": "NUMBER",
                                    "numDecimalDigits": -1,
                                    "currency": "$",
                                    "nanFormat": ""
                                },
                                "formatType": "NUMBER_FORMATTER"
                            }
                        }
                    }
                },
                {
                    "description": "shared tooltip layer",
                    "encoding": {
                        "x": {
                            "field": "Hour (ET)",
                            "type": "temporal",
                            "title": "",
                            "scale": {},
                            "axis": {
                                "grid": true,
                                "ticks": true,
                                "labels": true,
                                "labelFlush": false
                            }
                        }
                    },
                    "layer": [
                        {
                            "description": "shared tooltip rule mark layer",
                            "mark": {
                                "type": "rule",
                                "clip": true
                            },
                            "encoding": {
                                "opacity": {
                                    "condition": {
                                        "value": 0.3,
                                        "param": "shared_pivot_hover_param",
                                        "empty": false
                                    },
                                    "value": 0
                                },
                                "color": {
                                    "value": "CHART_DEFAULT_RULE_COLOR_MARKER"
                                },
                                "tooltip": [
                                    {
                                        "field": "Hour (ET)",
                                        "type": "temporal",
                                        "title": " "
                                    },
                                    {
                                        "field": "Baseline",
                                        "type": "quantitative",
                                        "aggregate": "sum",
                                        "formatType": "NUMBER_FORMATTER",
                                        "format": {
                                            "format": "NUMBER",
                                            "columnType": "NUMBER",
                                            "numDecimalDigits": -1,
                                            "currency": "$",
                                            "nanFormat": ""
                                        },
                                        "title": "Interior / Baseline"
                                    },
                                    {
                                        "field": "Temp: Outdoor - Control",
                                        "type": "quantitative",
                                        "formatType": "NUMBER_FORMATTER",
                                        "format": {
                                            "format": "NUMBER",
                                            "columnType": "NUMBER",
                                            "numDecimalDigits": -1,
                                            "currency": "$",
                                            "nanFormat": ""
                                        },
                                        "title": "Difference Between Interior Shady and Outdoor"
                                    },
                                    {
                                        "field": "Temp: Compare - Control",
                                        "type": "quantitative",
                                        "formatType": "NUMBER_FORMATTER",
                                        "format": {
                                            "format": "NUMBER",
                                            "columnType": "NUMBER",
                                            "numDecimalDigits": -1,
                                            "currency": "$",
                                            "nanFormat": ""
                                        },
                                        "title": "Difference Between Interior Shady and Intrerior Sunny"
                                    }
                                ]
                            },
                            "params": [
                                {
                                    "name": "shared_pivot_hover_param",
                                    "select": {
                                        "type": "point",
                                        "fields": [
                                            "Hour (ET)"
                                        ],
                                        "nearest": true,
                                        "on": "mouseover",
                                        "clear": "mouseout"
                                    }
                                }
                            ]
                        }
                    ]
                }
            ],
            "transform": [
                {
                    "calculate": "toDate(datum[\"Hour (ET)\"])",
                    "as": "Hour (ET)"
                },
                {
                    "filter": "isValid(datum[\"Hour (ET)\"])"
                }
            ]
        }
    ],
    "config": {
        "legend": {
            "orient": "bottom"
        },
        "font": "\"IBM Plex Sans\", system-ui, -apple-system, BlinkMacSystemFont, sans-serif",
        "view": {}
    },
    "datasets": {
        "layer00": [
            {
                "name": "dummy",
                "value": 0
            }
        ]
    },
    "usermeta": {
        "selectionConfigs": {},
        "columnNameMappings": {}
    }
}
""")
chart_compare_temp_HI_df.datasets.layer00 = compare_temp_HI_df.to_json(orient='records')
chart_compare_temp_HI_df.display(actions=False)

In [ ]:
import altair
chart_compare_temp_HI_df = altair.Chart.from_json(r"""
{
    "width": "container",
    "height": "container",
    "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
    "layer": [
        {
            "description": "outer data layer",
            "resolve": {
                "scale": {
                    "color": "independent",
                    "y": "shared"
                }
            },
            "layer": [
                {
                    "description": "line series layer",
                    "transform": [],
                    "layer": [
                        {
                            "description": "line mark layer",
                            "mark": {
                                "type": "line",
                                "point": false,
                                "clip": true,
                                "strokeWidth": 5,
                                "strokeCap": "square",
                                "strokeJoin": "round",
                                "cursor": "crosshair"
                            },
                            "encoding": {
                                "opacity": {
                                    "value": 1
                                }
                            }
                        },
                        {
                            "description": "pivot tooltip point mark layer",
                            "mark": {
                                "type": "point",
                                "tooltip": false,
                                "clip": true
                            },
                            "encoding": {
                                "opacity": {
                                    "value": 0,
                                    "condition": {
                                        "param": "shared_pivot_hover_param",
                                        "value": 1,
                                        "empty": false
                                    }
                                },
                                "size": {
                                    "value": 80
                                }
                            }
                        }
                    ],
                    "encoding": {
                        "x": {
                            "field": "Hour (ET)",
                            "type": "temporal",
                            "title": "",
                            "scale": {},
                            "axis": {
                                "grid": true,
                                "ticks": true,
                                "labels": true,
                                "labelFlush": false
                            }
                        },
                        "y": {
                            "field": "Baseline",
                            "type": "quantitative",
                            "title": "Degrees Difference (F)",
                            "scale": {
                                "domainMin": -14,
                                "domainMax": 24
                            },
                            "axis": {
                                "grid": true,
                                "ticks": true,
                                "labels": true,
                                "labelFlush": false,
                                "format": {
                                    "format": "NUMBER",
                                    "columnType": "NUMBER",
                                    "numDecimalDigits": -1,
                                    "currency": "$",
                                    "nanFormat": ""
                                },
                                "formatType": "NUMBER_FORMATTER"
                            }
                        },
                        "color": {
                            "datum": "Interior / Baseline",
                            "scale": {
                                "range": [
                                    "#4C78A8",
                                    "#4C78A8"
                                ]
                            },
                            "legend": {
                                "symbolOpacity": 1,
                                "symbolType": "stroke"
                            }
                        }
                    }
                },
                {
                    "description": "line series layer",
                    "transform": [],
                    "layer": [
                        {
                            "description": "line mark layer",
                            "mark": {
                                "type": "line",
                                "point": false,
                                "clip": true,
                                "strokeCap": "square",
                                "strokeJoin": "round",
                                "cursor": "crosshair"
                            },
                            "encoding": {
                                "opacity": {
                                    "value": 1
                                }
                            }
                        },
                        {
                            "description": "pivot tooltip point mark layer",
                            "mark": {
                                "type": "point",
                                "tooltip": false,
                                "clip": true
                            },
                            "encoding": {
                                "opacity": {
                                    "value": 0,
                                    "condition": {
                                        "param": "shared_pivot_hover_param",
                                        "value": 1,
                                        "empty": false
                                    }
                                },
                                "size": {
                                    "value": 80
                                }
                            }
                        }
                    ],
                    "encoding": {
                        "x": {
                            "field": "Hour (ET)",
                            "type": "temporal",
                            "title": "",
                            "scale": {},
                            "axis": {
                                "grid": true,
                                "ticks": true,
                                "labels": true,
                                "labelFlush": false
                            }
                        },
                        "y": {
                            "field": "Temp: Outdoor - Control",
                            "type": "quantitative",
                            "title": "Degrees Difference (F)",
                            "scale": {
                                "domainMin": -14,
                                "domainMax": 24
                            },
                            "axis": {
                                "grid": true,
                                "ticks": true,
                                "labels": true,
                                "labelFlush": false,
                                "format": {
                                    "format": "NUMBER",
                                    "columnType": "NUMBER",
                                    "numDecimalDigits": -1,
                                    "currency": "$",
                                    "nanFormat": ""
                                },
                                "formatType": "NUMBER_FORMATTER"
                            }
                        },
                        "color": {
                            "datum": "Difference Between Interior and Outdoor",
                            "scale": {
                                "range": [
                                    "#E45756",
                                    "#E45756"
                                ]
                            },
                            "legend": {
                                "symbolOpacity": 1,
                                "symbolType": "stroke"
                            }
                        }
                    }
                },
                {
                    "description": "line series layer",
                    "transform": [],
                    "layer": [
                        {
                            "description": "line mark layer",
                            "mark": {
                                "type": "line",
                                "point": false,
                                "clip": true,
                                "strokeCap": "square",
                                "strokeJoin": "round",
                                "cursor": "crosshair"
                            },
                            "encoding": {
                                "opacity": {
                                    "value": 1
                                }
                            }
                        },
                        {
                            "description": "pivot tooltip point mark layer",
                            "mark": {
                                "type": "point",
                                "tooltip": false,
                                "clip": true
                            },
                            "encoding": {
                                "opacity": {
                                    "value": 0,
                                    "condition": {
                                        "param": "shared_pivot_hover_param",
                                        "value": 1,
                                        "empty": false
                                    }
                                },
                                "size": {
                                    "value": 80
                                }
                            }
                        }
                    ],
                    "encoding": {
                        "x": {
                            "field": "Hour (ET)",
                            "type": "temporal",
                            "title": "",
                            "scale": {},
                            "axis": {
                                "grid": true,
                                "ticks": true,
                                "labels": true,
                                "labelFlush": false
                            }
                        },
                        "y": {
                            "field": "Temp: Outdoor - Compare",
                            "type": "quantitative",
                            "title": "Degrees Difference (F)",
                            "scale": {
                                "domainMin": -14,
                                "domainMax": 24
                            },
                            "axis": {
                                "grid": true,
                                "ticks": true,
                                "labels": true,
                                "labelFlush": false,
                                "format": {
                                    "format": "NUMBER",
                                    "columnType": "NUMBER",
                                    "numDecimalDigits": -1,
                                    "currency": "$",
                                    "nanFormat": ""
                                },
                                "formatType": "NUMBER_FORMATTER"
                            }
                        },
                        "color": {
                            "datum": "Difference Between Interior Sunny and Outdoor",
                            "scale": {
                                "range": [
                                    "#F58518",
                                    "#F58518"
                                ]
                            },
                            "legend": {
                                "symbolOpacity": 1,
                                "symbolType": "stroke"
                            }
                        }
                    }
                },
                {
                    "description": "shared tooltip layer",
                    "encoding": {
                        "x": {
                            "field": "Hour (ET)",
                            "type": "temporal",
                            "title": "",
                            "scale": {},
                            "axis": {
                                "grid": true,
                                "ticks": true,
                                "labels": true,
                                "labelFlush": false
                            }
                        }
                    },
                    "layer": [
                        {
                            "description": "shared tooltip rule mark layer",
                            "mark": {
                                "type": "rule",
                                "clip": true
                            },
                            "encoding": {
                                "opacity": {
                                    "condition": {
                                        "value": 0.3,
                                        "param": "shared_pivot_hover_param",
                                        "empty": false
                                    },
                                    "value": 0
                                },
                                "color": {
                                    "value": "CHART_DEFAULT_RULE_COLOR_MARKER"
                                },
                                "tooltip": [
                                    {
                                        "field": "Hour (ET)",
                                        "type": "temporal",
                                        "title": " "
                                    },
                                    {
                                        "field": "Baseline",
                                        "type": "quantitative",
                                        "formatType": "NUMBER_FORMATTER",
                                        "format": {
                                            "format": "NUMBER",
                                            "columnType": "NUMBER",
                                            "numDecimalDigits": -1,
                                            "currency": "$",
                                            "nanFormat": ""
                                        },
                                        "title": "Interior / Baseline"
                                    },
                                    {
                                        "field": "Temp: Outdoor - Control",
                                        "type": "quantitative",
                                        "formatType": "NUMBER_FORMATTER",
                                        "format": {
                                            "format": "NUMBER",
                                            "columnType": "NUMBER",
                                            "numDecimalDigits": -1,
                                            "currency": "$",
                                            "nanFormat": ""
                                        },
                                        "title": "Difference Between Interior and Outdoor"
                                    },
                                    {
                                        "field": "Temp: Outdoor - Compare",
                                        "type": "quantitative",
                                        "formatType": "NUMBER_FORMATTER",
                                        "format": {
                                            "format": "NUMBER",
                                            "columnType": "NUMBER",
                                            "numDecimalDigits": -1,
                                            "currency": "$",
                                            "nanFormat": ""
                                        },
                                        "title": "Difference Between Interior Sunny and Outdoor"
                                    }
                                ]
                            },
                            "params": [
                                {
                                    "name": "shared_pivot_hover_param",
                                    "select": {
                                        "type": "point",
                                        "fields": [
                                            "Hour (ET)"
                                        ],
                                        "nearest": true,
                                        "on": "mouseover",
                                        "clear": "mouseout"
                                    }
                                }
                            ]
                        }
                    ]
                }
            ],
            "transform": [
                {
                    "calculate": "toDate(datum[\"Hour (ET)\"])",
                    "as": "Hour (ET)"
                },
                {
                    "filter": "isValid(datum[\"Hour (ET)\"])"
                }
            ]
        }
    ],
    "config": {
        "legend": {
            "orient": "bottom"
        },
        "font": "\"IBM Plex Sans\", system-ui, -apple-system, BlinkMacSystemFont, sans-serif",
        "view": {}
    },
    "datasets": {
        "layer00": [
            {
                "name": "dummy",
                "value": 0
            }
        ]
    },
    "usermeta": {
        "selectionConfigs": {},
        "columnNameMappings": {}
    }
}
""")
chart_compare_temp_HI_df.datasets.layer00 = compare_temp_HI_df.to_json(orient='records')
chart_compare_temp_HI_df.display(actions=False)

#### It's Not the Heat... It's the Humidity!
Indeed, when examining the heat index data, the impact of the shades is even more dramatic:

In [ ]:
import altair
chart_compare_temp_HI_df = altair.Chart.from_json(r"""
{
    "width": "container",
    "height": "container",
    "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
    "layer": [
        {
            "description": "outer data layer",
            "resolve": {
                "scale": {
                    "color": "independent",
                    "y": "shared"
                }
            },
            "layer": [
                {
                    "description": "line series layer",
                    "transform": [],
                    "layer": [
                        {
                            "description": "line mark layer",
                            "mark": {
                                "type": "line",
                                "point": false,
                                "clip": true,
                                "strokeWidth": 5,
                                "strokeCap": "square",
                                "strokeJoin": "round",
                                "cursor": "crosshair"
                            },
                            "encoding": {
                                "opacity": {
                                    "value": 1
                                }
                            }
                        },
                        {
                            "description": "pivot tooltip point mark layer",
                            "mark": {
                                "type": "point",
                                "tooltip": false,
                                "clip": true
                            },
                            "encoding": {
                                "opacity": {
                                    "value": 0,
                                    "condition": {
                                        "param": "shared_pivot_hover_param",
                                        "value": 1,
                                        "empty": false
                                    }
                                },
                                "size": {
                                    "value": 80
                                }
                            }
                        }
                    ],
                    "encoding": {
                        "x": {
                            "field": "Hour (ET)",
                            "type": "temporal",
                            "title": "",
                            "scale": {},
                            "axis": {
                                "grid": true,
                                "ticks": true,
                                "labels": true,
                                "labelFlush": false
                            }
                        },
                        "y": {
                            "field": "Baseline",
                            "type": "quantitative",
                            "title": "Degrees Difference (F)",
                            "scale": {},
                            "axis": {
                                "grid": true,
                                "ticks": true,
                                "labels": true,
                                "labelFlush": false,
                                "format": {
                                    "format": "NUMBER",
                                    "columnType": "NUMBER",
                                    "numDecimalDigits": -1,
                                    "currency": "$",
                                    "nanFormat": ""
                                },
                                "formatType": "NUMBER_FORMATTER"
                            }
                        },
                        "color": {
                            "datum": "Interior / Baseline",
                            "scale": {
                                "range": [
                                    "#4C78A8",
                                    "#4C78A8"
                                ]
                            },
                            "legend": {
                                "symbolOpacity": 1,
                                "symbolType": "stroke"
                            }
                        }
                    }
                },
                {
                    "description": "area series layer",
                    "transform": [],
                    "layer": [
                        {
                            "description": "background area mark layer",
                            "mark": {
                                "type": "area",
                                "point": false,
                                "line": true,
                                "strokeJoin": "round",
                                "clip": true,
                                "cursor": "crosshair"
                            },
                            "encoding": {
                                "color": {
                                    "datum": "Difference Between Interior Shady and Outdoor",
                                    "scale": {
                                        "range": [
                                            "#E45756",
                                            "#E45756"
                                        ]
                                    },
                                    "legend": {
                                        "symbolOpacity": 1
                                    }
                                },
                                "opacity": {
                                    "value": 0.2
                                }
                            }
                        },
                        {
                            "transform": [],
                            "description": "area mark layer",
                            "mark": {
                                "type": "area",
                                "point": false,
                                "line": true,
                                "clip": true
                            },
                            "encoding": {
                                "color": {
                                    "datum": "Difference Between Interior Shady and Outdoor",
                                    "scale": {
                                        "range": [
                                            "#E45756",
                                            "#E45756"
                                        ]
                                    },
                                    "legend": {
                                        "symbolOpacity": 1
                                    }
                                }
                            }
                        },
                        {
                            "description": "pivot tooltip point mark layer",
                            "mark": {
                                "type": "point",
                                "tooltip": false,
                                "clip": true
                            },
                            "encoding": {
                                "opacity": {
                                    "value": 0,
                                    "condition": {
                                        "param": "shared_pivot_hover_param",
                                        "value": 1,
                                        "empty": false
                                    }
                                },
                                "size": {
                                    "value": 80
                                },
                                "color": {
                                    "datum": "Difference Between Interior Shady and Outdoor",
                                    "scale": {
                                        "range": [
                                            "#E45756",
                                            "#E45756"
                                        ]
                                    },
                                    "legend": {
                                        "symbolOpacity": 1
                                    }
                                },
                                "y": {
                                    "field": "HI: Outdoor - Control",
                                    "type": "quantitative",
                                    "title": "Degrees Difference (F)",
                                    "scale": {},
                                    "axis": {
                                        "grid": true,
                                        "ticks": true,
                                        "labels": true,
                                        "labelFlush": false,
                                        "format": {
                                            "format": "NUMBER",
                                            "columnType": "NUMBER",
                                            "numDecimalDigits": -1,
                                            "currency": "$",
                                            "nanFormat": ""
                                        },
                                        "formatType": "NUMBER_FORMATTER"
                                    },
                                    "stack": "zero"
                                }
                            }
                        }
                    ],
                    "encoding": {
                        "x": {
                            "field": "Hour (ET)",
                            "type": "temporal",
                            "title": "",
                            "scale": {},
                            "axis": {
                                "grid": true,
                                "ticks": true,
                                "labels": true,
                                "labelFlush": false
                            }
                        },
                        "y": {
                            "field": "HI: Outdoor - Control",
                            "type": "quantitative",
                            "title": "Degrees Difference (F)",
                            "scale": {},
                            "axis": {
                                "grid": true,
                                "ticks": true,
                                "labels": true,
                                "labelFlush": false,
                                "format": {
                                    "format": "NUMBER",
                                    "columnType": "NUMBER",
                                    "numDecimalDigits": -1,
                                    "currency": "$",
                                    "nanFormat": ""
                                },
                                "formatType": "NUMBER_FORMATTER"
                            }
                        }
                    }
                },
                {
                    "description": "area series layer",
                    "transform": [],
                    "layer": [
                        {
                            "description": "background area mark layer",
                            "mark": {
                                "type": "area",
                                "point": false,
                                "line": true,
                                "strokeJoin": "round",
                                "clip": true,
                                "cursor": "crosshair"
                            },
                            "encoding": {
                                "color": {
                                    "datum": "Difference Between Interior Shady and Intrerior Sunny",
                                    "scale": {
                                        "range": [
                                            "#F58518",
                                            "#F58518"
                                        ]
                                    },
                                    "legend": {
                                        "symbolOpacity": 1
                                    }
                                },
                                "opacity": {
                                    "value": 0.2
                                }
                            }
                        },
                        {
                            "transform": [],
                            "description": "area mark layer",
                            "mark": {
                                "type": "area",
                                "point": false,
                                "line": true,
                                "clip": true
                            },
                            "encoding": {
                                "color": {
                                    "datum": "Difference Between Interior Shady and Intrerior Sunny",
                                    "scale": {
                                        "range": [
                                            "#F58518",
                                            "#F58518"
                                        ]
                                    },
                                    "legend": {
                                        "symbolOpacity": 1
                                    }
                                }
                            }
                        },
                        {
                            "description": "pivot tooltip point mark layer",
                            "mark": {
                                "type": "point",
                                "tooltip": false,
                                "clip": true
                            },
                            "encoding": {
                                "opacity": {
                                    "value": 0,
                                    "condition": {
                                        "param": "shared_pivot_hover_param",
                                        "value": 1,
                                        "empty": false
                                    }
                                },
                                "size": {
                                    "value": 80
                                },
                                "color": {
                                    "datum": "Difference Between Interior Shady and Intrerior Sunny",
                                    "scale": {
                                        "range": [
                                            "#F58518",
                                            "#F58518"
                                        ]
                                    },
                                    "legend": {
                                        "symbolOpacity": 1
                                    }
                                },
                                "y": {
                                    "field": "HI: Compare - Control",
                                    "type": "quantitative",
                                    "title": "Degrees Difference (F)",
                                    "scale": {},
                                    "axis": {
                                        "grid": true,
                                        "ticks": true,
                                        "labels": true,
                                        "labelFlush": false,
                                        "format": {
                                            "format": "NUMBER",
                                            "columnType": "NUMBER",
                                            "numDecimalDigits": -1,
                                            "currency": "$",
                                            "nanFormat": ""
                                        },
                                        "formatType": "NUMBER_FORMATTER"
                                    },
                                    "stack": "zero"
                                }
                            }
                        }
                    ],
                    "encoding": {
                        "x": {
                            "field": "Hour (ET)",
                            "type": "temporal",
                            "title": "",
                            "scale": {},
                            "axis": {
                                "grid": true,
                                "ticks": true,
                                "labels": true,
                                "labelFlush": false
                            }
                        },
                        "y": {
                            "field": "HI: Compare - Control",
                            "type": "quantitative",
                            "title": "Degrees Difference (F)",
                            "scale": {},
                            "axis": {
                                "grid": true,
                                "ticks": true,
                                "labels": true,
                                "labelFlush": false,
                                "format": {
                                    "format": "NUMBER",
                                    "columnType": "NUMBER",
                                    "numDecimalDigits": -1,
                                    "currency": "$",
                                    "nanFormat": ""
                                },
                                "formatType": "NUMBER_FORMATTER"
                            }
                        }
                    }
                },
                {
                    "description": "shared tooltip layer",
                    "encoding": {
                        "x": {
                            "field": "Hour (ET)",
                            "type": "temporal",
                            "title": "",
                            "scale": {},
                            "axis": {
                                "grid": true,
                                "ticks": true,
                                "labels": true,
                                "labelFlush": false
                            }
                        }
                    },
                    "layer": [
                        {
                            "description": "shared tooltip rule mark layer",
                            "mark": {
                                "type": "rule",
                                "clip": true
                            },
                            "encoding": {
                                "opacity": {
                                    "condition": {
                                        "value": 0.3,
                                        "param": "shared_pivot_hover_param",
                                        "empty": false
                                    },
                                    "value": 0
                                },
                                "color": {
                                    "value": "CHART_DEFAULT_RULE_COLOR_MARKER"
                                },
                                "tooltip": [
                                    {
                                        "field": "Hour (ET)",
                                        "type": "temporal",
                                        "title": " "
                                    },
                                    {
                                        "field": "Baseline",
                                        "type": "quantitative",
                                        "formatType": "NUMBER_FORMATTER",
                                        "format": {
                                            "format": "NUMBER",
                                            "columnType": "NUMBER",
                                            "numDecimalDigits": -1,
                                            "currency": "$",
                                            "nanFormat": ""
                                        },
                                        "title": "Interior / Baseline"
                                    },
                                    {
                                        "field": "HI: Outdoor - Control",
                                        "type": "quantitative",
                                        "formatType": "NUMBER_FORMATTER",
                                        "format": {
                                            "format": "NUMBER",
                                            "columnType": "NUMBER",
                                            "numDecimalDigits": -1,
                                            "currency": "$",
                                            "nanFormat": ""
                                        },
                                        "title": "Difference Between Interior Shady and Outdoor"
                                    },
                                    {
                                        "field": "HI: Compare - Control",
                                        "type": "quantitative",
                                        "formatType": "NUMBER_FORMATTER",
                                        "format": {
                                            "format": "NUMBER",
                                            "columnType": "NUMBER",
                                            "numDecimalDigits": -1,
                                            "currency": "$",
                                            "nanFormat": ""
                                        },
                                        "title": "Difference Between Interior Shady and Intrerior Sunny"
                                    }
                                ]
                            },
                            "params": [
                                {
                                    "name": "shared_pivot_hover_param",
                                    "select": {
                                        "type": "point",
                                        "fields": [
                                            "Hour (ET)"
                                        ],
                                        "nearest": true,
                                        "on": "mouseover",
                                        "clear": "mouseout"
                                    }
                                }
                            ]
                        }
                    ]
                }
            ],
            "transform": [
                {
                    "calculate": "toDate(datum[\"Hour (ET)\"])",
                    "as": "Hour (ET)"
                },
                {
                    "filter": "isValid(datum[\"Hour (ET)\"])"
                }
            ]
        }
    ],
    "config": {
        "legend": {
            "orient": "bottom"
        },
        "font": "\"IBM Plex Sans\", system-ui, -apple-system, BlinkMacSystemFont, sans-serif",
        "view": {}
    },
    "datasets": {
        "layer00": [
            {
                "name": "dummy",
                "value": 0
            }
        ]
    },
    "usermeta": {
        "selectionConfigs": {},
        "columnNameMappings": {}
    }
}
""")
chart_compare_temp_HI_df.datasets.layer00 = compare_temp_HI_df.to_json(orient='records')
chart_compare_temp_HI_df.display(actions=False)

In [ ]:
# Reduce data to examine peak sunny periods only
peak_sun_start = pd.to_datetime('11:00:00').time()
peak_sun_end = pd.to_datetime('19:00:00').time()

peak_sun_temp_df = compare_temp_HI_df.copy()
peak_sun_temp_df = peak_sun_temp_df.drop(columns=['Baseline'])
peak_sun_temp_df['time'] = peak_sun_temp_df['Hour (ET)'].dt.time
peak_sun_temp_df = peak_sun_temp_df[(peak_sun_temp_df['time'] >= peak_sun_start) & (peak_sun_temp_df['time'] <= peak_sun_end)]

# Reduce data further to when the Local Outdoor Temperature is greater than the threshold emperature
THRESHOLD_TEMP = 80.0
peak_sun_temp_df = peak_sun_temp_df[(peak_sun_temp_df['Outdoor Temp (F)'] > THRESHOLD_TEMP)]

peak_sun_temp_df


,Hour (ET),Control Temp (F),Compare Temp (F),Outdoor Temp (F),Control Heat Index (F),Compare Heat Index (F),Outdoor Heat Index (F),Temp: Compare - Control,Temp: Outdoor - Control,Temp: Outdoor - Compare,HI: Compare - Control,HI: Outdoor - Control,HI: Outdoor - Compare,time
1,2024-08-12 18:00:00,80.07188,81.80798,80.5,82.3,84.2,80.8,1.74,0.43,-1.31,1.9,-1.5,-3.4,18:00:00
3,2024-08-12 16:00:00,80.11796,82.10012,80.5,82.4,84.6,81.2,1.98,0.38,-1.60,2.2,-1.2,-3.4,16:00:00
4,2024-08-12 15:00:00,79.94750,83.62526,82.6,82.2,86.4,82.4,3.68,2.65,-1.03,4.2,0.2,-4.0,15:00:00
5,2024-08-12 14:00:00,79.31534,85.28486,82.3,79.9,88.2,82.0,5.97,2.98,-2.98,8.3,2.1,-6.2,14:00:00
6,2024-08-12 13:00:00,78.56834,84.94178,81.0,79.1,87.4,81.3,6.37,2.43,-3.94,8.3,2.2,-6.1,13:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
410,2024-07-26 13:00:00,77.21240,81.84524,82.7,77.6,83.5,82.5,4.63,5.49,0.85,5.9,4.9,-1.0,13:00:00
411,2024-07-26 12:00:00,76.97264,80.08358,80.5,77.3,81.6,80.7,3.11,3.53,0.42,4.3,3.4,-0.9,12:00:00
428,2024-07-25 19:00:00,80.26574,81.44636,82.1,83.0,84.4,82.6,1.18,1.83,0.65,1.4,-0.4,-1.8,19:00:00
429,2024-07-25 18:00:00,80.33954,82.16024,82.2,83.4,85.9,83.3,1.82,1.86,0.04,2.5,-0.1,-2.6,18:00:00


## Conclusion: The Bright Side of Shade



In [ ]:
# Calculate overall average, range,and avg during afternoons, above threshold temp
def diffs_stats(temp_data, data_HI):

    return [
            {
            'Average Temperature Difference': temp_data.mean().round(1),
            'Average Heat Index Difference': data_HI.mean().round(1),
            'Max Temperature Difference': round(temp_data.max(), 1),
            'Max Heat Index Difference': round(data_HI.max(), 1)
        }
    ]
diffs_compare = diffs_stats(peak_sun_temp_df.loc[:, 'Temp: Outdoor - Compare'], peak_sun_temp_df.loc[:, 'HI: Outdoor - Compare'])
diffs_outdoor = diffs_stats(peak_sun_temp_df.loc[:, 'Temp: Outdoor - Control'], peak_sun_temp_df.loc[:, 'HI: Outdoor - Control'])

diffs_summary = []
diffs_summary.extend(diffs_outdoor)
diffs_summary.extend(diffs_compare)
diffs_summary_df = pd.DataFrame(diffs_summary, index=['Shady Interior vs Outdoors (°F)', 'Sunny Interior vs Outdoors (°F)'])
# diffs_summary_df
diffs_summary_display_df = diffs_summary_df.transpose()
diffs_summary_display_df

,Shady Interior vs Outdoors (°F),Sunny Interior vs Outdoors (°F)
Average Temperature Difference,4.6,1.2
Average Heat Index Difference,6.7,2.0
Max Temperature Difference,14.2,10.6
Max Heat Index Difference,23.1,17.4


In [ ]:
avg_diff_outdoor_temp = diffs_summary_df.loc['Shady Interior vs Outdoors (°F)', 'Average Temperature Difference']
avg_diff_sunny_temp = diffs_summary_df.loc['Sunny Interior vs Outdoors (°F)', 'Average Temperature Difference']
avg_diff_outdoor_HI = diffs_summary_df.loc['Shady Interior vs Outdoors (°F)', 'Average Heat Index Difference']
avg_diff_sunny_HI = diffs_summary_df.loc['Sunny Interior vs Outdoors (°F)', 'Average Heat Index Difference']
max_diff_outdoor_temp = diffs_summary_df.loc['Shady Interior vs Outdoors (°F)', 'Max Temperature Difference']
max_diff_outdoor_HI = diffs_summary_df.loc['Shady Interior vs Outdoors (°F)', 'Max Heat Index Difference']

display_summary = (f'''During afternoons with an outdoor temperature above {THRESHOLD_TEMP:.0f}°F:
* On average, the closed shades reduced the temperature by {avg_diff_outdoor_temp:.1f}°F and improved the heat index by {avg_diff_outdoor_HI:.1f}°F.
* On average, with open blinds, the temperature was only {avg_diff_sunny_temp:.1f}°F cooler than the outdoors, and a heat index decrease of {avg_diff_sunny_HI:.1f}°F.
* On the hottest day, the closed shades provided a {max_diff_outdoor_temp:.1f}°F reduction in temperature and {max_diff_outdoor_HI:.1f}°F drop in heat index.
''')
print(display_summary)


During afternoons with an outdoor temperature above 80°F:
* On average, the closed shades reduced the temperature by 4.6°F and improved the heat index by 6.7°F.
* On average, with open blinds, the temperature was only 1.2°F cooler than the outdoors, and a heat index decrease of 2.0°F.
* On the hottest day, the closed shades provided a 14.2°F reduction in temperature and 23.1°F drop in heat index.



During afternoons with an outdoor temperature above 80°F:
* On average, the closed shades reduced the temperature by 4.6°F and improved the heat index by 6.7°F.
* On average, with open blinds, the temperature was only 1.2°F cooler than the outdoors, and a heat index decrease of 2.0°F.
* On the hottest day, the closed shades provided a 14.2°F reduction in temperature and 23.1°F drop in heat index.

<br>

## Guiding Lights: Sources & Credits
### Data Sources
#### Temperature/humidity/pressure sensor
* Adafruit BME280 I2C or SPI Temperature Humidity Pressure Sensor - STEMMA QT
* https://www.adafruit.com/product/2652

#### CO2, Temperature, & humidity sensor
* Adafruit SCD-40 - True CO2, Temperature and Humidity Sensor - STEMMA QT / Qwiic
* https://www.adafruit.com/product/5187

#### Weather Data from Open-Meteo
* https://open-meteo.com/en/docs


### Formulas
#### Heat Index from NOAA, National Weather Service 
* https://www.wpc.ncep.noaa.gov/html/heatindex_equation.shtml
